In [4]:
from MasterDatabaseManagement.tools.database_management_helpers import remove_tz
from core.db import get_engine
from core.libs import pd, Path
from core.paths import DATABASE_EXPORTS_DIR, ensure_all_paths_exist
from MasterDatabaseManagement.Queries import fetch_active_contracts_query_df

# --- CONFIGURACIÓN ---
engine = get_engine()
ensure_all_paths_exist()

WANTED = [
    ("contract_tree_information",   "CTI"),
    ("farmer_personal_information", "FPI"),
    ("contract_allocation",         "CA"),
    ("inventory_metrics_current",   "IMC"),
    ("survival_current",            "Survival_Current"),
]

outfile = Path(DATABASE_EXPORTS_DIR) / "masterdatabase_export_joined.xlsx"

# --- FUNCIONES ---
def read_table(table):
    """Lee una tabla del esquema masterdatabase y remueve tz."""
    df = pd.read_sql(f'SELECT * FROM masterdatabase."{table}"', engine)
    return remove_tz(df)

# --- INICIO ---
print("📦 Iniciando export...")

tables = pd.read_sql("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'masterdatabase'
""", engine)["table_name"].str.lower().tolist()
existing = set(tables)


# --- EXPORT Y UNIÓN ---
joined_df = None
with pd.ExcelWriter(outfile, engine="openpyxl") as writer:

    # 1️⃣ Exportar tablas individuales
    dfs = {}
    for table, sheet_name in WANTED:
        if table.lower() not in existing:
            print(f"⚠️ Tabla no encontrada: {table}")
            continue
        df = read_table(table)
        dfs[sheet_name] = df
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"📄 Exportado: {sheet_name} ({len(df)} filas)")

    # 2️⃣ Active Farmers Query (final)
    try:
        afq_df = fetch_active_contracts_query_df(engine)
        afq_df = remove_tz(afq_df)
        afq_df.to_excel(writer, sheet_name="Active Farmers Query", index=False)
        print("✅ AFQ exportado.")
    except Exception as e:
        pd.DataFrame({"error": [str(e)]}).to_excel(
            writer, sheet_name="Active Farmers Query (error)", index=False
        )

    # 3️⃣ JOINS: expandir contract_codes de FPI
    try:
        fpi = dfs.get("FPI")
        if fpi is None:
            raise ValueError("FPI no encontrada, no se puede generar join")

        # expandir contract_codes (array de texto)
        fpi_exploded = fpi.explode("contract_codes").rename(columns={"contract_codes": "contract_code"})
        print(f"🔍 FPI expandida a {len(fpi_exploded)} filas.")

        # base del join
        joined_df = fpi_exploded.copy()

        # merge progresivo
        if "CTI" in dfs:
            joined_df = joined_df.merge(dfs["CTI"], on="contract_code", how="left", suffixes=("", "_CTI"))
        if "CA" in dfs:
            joined_df = joined_df.merge(dfs["CA"], on="contract_code", how="left", suffixes=("", "_CA"))
        if "IMC" in dfs:
            joined_df = joined_df.merge(dfs["IMC"], on="contract_code", how="left", suffixes=("", "_IMC"))
        if "Survival_Current" in dfs:
            joined_df = joined_df.merge(dfs["Survival_Current"], on="contract_code", how="left", suffixes=("", "_Surv"))

        # ordenar por farmer_number + contract_code
        if "farmer_number" in joined_df.columns and "contract_code" in joined_df.columns:
            joined_df = joined_df.sort_values(["farmer_number", "contract_code"])

        joined_df.to_excel(writer, sheet_name="MASTER_JOINED", index=False)
        print(f"🧩 MASTER_JOINED listo ({len(joined_df)} filas).")

    except Exception as e:
        pd.DataFrame({"error": [str(e)]}).to_excel(
            writer, sheet_name="MASTER_JOINED (error)", index=False
        )
        print("❌ Error al generar MASTER_JOINED:", e)

print(f"\n✅ Export final listo en: {outfile}")

📄 Exportado: CTI (462 filas)
📄 Exportado: FPI (385 filas)
📄 Exportado: CA (462 filas)
📄 Exportado: IMC (521 filas)
📄 Exportado: Survival_Current (428 filas)
✅ AFQ exportado.
🔍 FPI expandida a 461 filas.
🧩 MASTER_JOINED listo (556 filas).

✅ Export final listo en: C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\masterdatabase_export_joined.xlsx


In [5]:
from core.db import get_engine
from core.libs import pd

engine = get_engine()

# --- CARGA ---
imc = pd.read_sql('SELECT * FROM masterdatabase.inventory_metrics_current', engine)
im  = pd.read_sql('SELECT * FROM masterdatabase.inventory_metrics', engine)

print("📦 Totales:")
print(f" - inventory_metrics_current: {len(imc)} filas")
print(f" - inventory_metrics: {len(im)} filas")

# --- AGRUPACIÓN POR CONTRACT_CODE ---
grp_imc = imc.groupby("contract_code", dropna=False).size().reset_index(name="count_imc")
grp_im  = im.groupby("contract_code", dropna=False).size().reset_index(name="count_im")

# --- JOIN DE COMPARACIÓN ---
merged = grp_im.merge(grp_imc, on="contract_code", how="outer")

# Añadimos también el año de inventario (promedio o más reciente)
def summarize_years(df, label):
    return (
        df.groupby("contract_code")["inventory_year"]
        .agg(["min", "max"])
        .rename(columns={"min": f"{label}_min_year", "max": f"{label}_max_year"})
        .reset_index()
    )

years_im  = summarize_years(im, "IM")
years_imc = summarize_years(imc, "IMC")

# Unimos todo
merged = (
    merged.merge(years_im, on="contract_code", how="left")
           .merge(years_imc, on="contract_code", how="left")
)

# --- DIFERENCIAS ---
merged["count_diff"] = merged["count_imc"].fillna(0) - merged["count_im"].fillna(0)
merged["year_diff"]  = merged["IMC_max_year"] - merged["IM_max_year"]

# --- RESULTADOS ---
print("\n📊 Resumen general:")
print(merged.describe(include="all"))

# Contratos con discrepancias en conteo
diff_counts = merged[merged["count_diff"] != 0]
print(f"\n⚠️ Contratos con diferencia de registros: {len(diff_counts)}")

# Contratos donde el año no coincide
diff_years = merged[merged["year_diff"].fillna(0) != 0]
print(f"📅 Contratos con diferencia en años de inventario: {len(diff_years)}")

# --- EXPORT A EXCEL ---
out = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_metrics_comparison.xlsx"
with pd.ExcelWriter(out, engine="openpyxl") as writer:
    im.to_excel(writer, sheet_name="Inventory_Metrics", index=False)
    imc.to_excel(writer, sheet_name="Inventory_Metrics_Current", index=False)
    merged.to_excel(writer, sheet_name="Comparison", index=False)

print(f"\n✅ Comparación exportada: {out}")


💻 Conectado a la base de datos helloworldtree
📦 Totales:
 - inventory_metrics_current: 521 filas
 - inventory_metrics: 295 filas

📊 Resumen general:
       contract_code    count_im   count_imc  IM_min_year  IM_max_year  \
count            424  198.000000  424.000000   198.000000   198.000000   
unique           424         NaN         NaN          NaN          NaN   
top           US0165         NaN         NaN          NaN          NaN   
freq               1         NaN         NaN          NaN          NaN   
mean             NaN    1.489899    1.228774  2022.868687  2023.823232   
std              NaN    0.752255    0.568705     1.704854     1.419253   
min              NaN    1.000000    1.000000  2021.000000  2021.000000   
25%              NaN    1.000000    1.000000  2021.000000  2022.500000   
50%              NaN    1.000000    1.000000  2024.000000  2024.000000   
75%              NaN    2.000000    1.000000  2024.000000  2025.000000   
max              NaN    4.000000    4

In [6]:
from core.db import get_engine
from core.libs import pd

engine = get_engine()
count = pd.read_sql('SELECT COUNT(*) AS total FROM masterdatabase.contract_tree_information', engine)
print(count)


💻 Conectado a la base de datos helloworldtree
   total
0    462


In [7]:
from core.db import get_engine
from core.libs import pd

engine = get_engine()

# --- CARGA ---
cti = pd.read_sql('SELECT contract_code FROM masterdatabase.contract_tree_information', engine)
imc = pd.read_sql('SELECT contract_code, inventory_year FROM masterdatabase.inventory_metrics_current', engine)
im  = pd.read_sql('SELECT contract_code, inventory_year FROM masterdatabase.inventory_metrics', engine)

print(f"📦 Registros cargados:")
print(f" - CTI: {len(cti)} contratos")
print(f" - IMC: {len(imc)} inventarios actuales")
print(f" - IM:  {len(im)} inventarios históricos")

# --- NORMALIZAR ---
# for df in [cti, imc, im]:
#     df["contract_code"] = df["contract_code"].astype(str).str.strip()

# --- CONTRATOS FALTANTES ---
missing_in_imc = cti[~cti["contract_code"].isin(imc["contract_code"])]
missing_in_im  = cti[~cti["contract_code"].isin(im["contract_code"])]

print(f"\n🚫 En CTI pero no en IMC: {len(missing_in_imc)}")
print(f"🚫 En CTI pero no en IM:  {len(missing_in_im)}")

# --- DUPLICADOS Y AGRUPADOS POR AÑO ---
def detect_duplicates(df, label):
    dup = (
        df.groupby("contract_code")
        .agg(count=("inventory_year", "count"),
             years=("inventory_year", lambda x: sorted(set(x))))
        .reset_index()
    )
    dup = dup[dup["count"] > 1].sort_values("count", ascending=False)
    dup["source"] = label
    return dup

dups_imc = detect_duplicates(imc, "IMC")
dups_im  = detect_duplicates(im, "IM")

print(f"\n⚠️ Contratos duplicados en IMC: {len(dups_imc)}")
print(f"⚠️ Contratos duplicados en IM:  {len(dups_im)}")

# --- EXPORT ---
out = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_verification_vs_cti.xlsx"
with pd.ExcelWriter(out, engine="openpyxl") as writer:
    missing_in_imc.to_excel(writer, sheet_name="Missing_in_IMC", index=False)
    missing_in_im.to_excel(writer, sheet_name="Missing_in_IM", index=False)
    dups_imc.to_excel(writer, sheet_name="Duplicates_IMC", index=False)
    dups_im.to_excel(writer, sheet_name="Duplicates_IM", index=False)

print(f"\n✅ Verificación exportada a: {out}")


💻 Conectado a la base de datos helloworldtree
📦 Registros cargados:
 - CTI: 462 contratos
 - IMC: 521 inventarios actuales
 - IM:  295 inventarios históricos

🚫 En CTI pero no en IMC: 40
🚫 En CTI pero no en IM:  266

⚠️ Contratos duplicados en IMC: 70
⚠️ Contratos duplicados en IM:  70

✅ Verificación exportada a: C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_verification_vs_cti.xlsx


In [8]:
from core.db import get_engine
from core.libs import pd

engine = get_engine()

# --- CARGA ---
imc = pd.read_sql('SELECT * FROM masterdatabase.inventory_metrics_current', engine)
print(f"📦 IMC cargado: {len(imc)} filas totales")

# --- VERIFICAR CAMPOS DISPONIBLES ---
if "contract_code" not in imc.columns or "inventory_year" not in imc.columns:
    raise ValueError("Faltan columnas necesarias: 'contract_code' o 'inventory_year'")

# --- IDENTIFICAR DUPLICADOS ---
dups = (
    imc.groupby("contract_code", dropna=False)["inventory_year"]
    .agg(list)
    .reset_index()
)
dups["count"] = dups["inventory_year"].apply(len)
dups = dups[dups["count"] > 1]
print(f"⚠️ Contratos con múltiples años: {len(dups)}")

# --- FILTRAR SOLO LOS MÁS RECIENTES ---
# importante: no se normaliza texto, se usa el valor exacto de contract_code
latest_idx = (
    imc.sort_values("inventory_year", ascending=False)
       .groupby("contract_code", dropna=False)
       .head(1)
       .index
)
imc_latest = imc.loc[latest_idx]

print(f"✅ IMC depurado: {len(imc_latest)} filas (solo años más recientes)")


💻 Conectado a la base de datos helloworldtree
📦 IMC cargado: 521 filas totales
⚠️ Contratos con múltiples años: 70
✅ IMC depurado: 424 filas (solo años más recientes)


In [9]:
# --- EXPORT ---
out = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_metrics_current_latest.xlsx"
with pd.ExcelWriter(out, engine="openpyxl") as writer:
    imc.to_excel(writer, sheet_name="Original_IMC", index=False)
    imc_latest.to_excel(writer, sheet_name="Latest_Only", index=False)
    dups.to_excel(writer, sheet_name="Duplicated_Contracts", index=False)

print(f"\n📁 Export listo: {out}")



📁 Export listo: C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_metrics_current_latest.xlsx


In [16]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

# --- RESPALDO ---
print("📦 Creando respaldo temporal de masterdatabase.inventory_metrics_current...")

with engine.begin() as conn:
    # eliminar si ya existe
    conn.execute(text("DROP TABLE IF EXISTS masterdatabase.inventory_metrics_current_backup"))
    # crear copia exacta
    conn.execute(text("""
        CREATE TABLE masterdatabase.inventory_metrics_current_backup AS
        TABLE masterdatabase.inventory_metrics_current
    """))

print("✅ Respaldo creado: masterdatabase.inventory_metrics_current_backup")

# --- OBTENER MÁS RECIENTES ---
query_latest = """
WITH ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY contract_code ORDER BY inventory_year DESC) AS _rank_filter
    FROM masterdatabase.inventory_metrics_current
)
SELECT * FROM ranked WHERE _rank_filter = 1;
"""

latest_df = pd.read_sql(query_latest, engine)
print(f"📊 Filtrados {len(latest_df)} registros más recientes de IMC")

# Eliminar columnas auxiliares que no existen en la tabla destino
columns_to_drop = ['_rank_filter', 'rn']  # ajusta según tus columnas auxiliares
latest_df = latest_df.drop(columns=[col for col in columns_to_drop if col in latest_df.columns])

# --- REEMPLAZAR TABLA ORIGINAL CON FILTRADOS ---
# CAMBIO CLAVE: usa 'engine' directamente, no conn.connection
with engine.begin() as conn:
    conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics_current"))

# Escribir fuera del contexto de transacción para evitar conflictos
latest_df.to_sql(
    "inventory_metrics_current",
    con=engine,  # ← USA EL ENGINE DIRECTAMENTE
    schema="masterdatabase",
    if_exists="append",
    index=False
)

print("✅ Tabla masterdatabase.inventory_metrics_current actualizada con solo registros más recientes.")

# --- VALIDACIÓN FINAL ---
count_check = pd.read_sql("SELECT COUNT(*) AS total FROM masterdatabase.inventory_metrics_current", engine)
print(f"🔍 Registros finales en IMC: {count_check.loc[0, 'total']}")

💻 Conectado a la base de datos helloworldtree
📦 Creando respaldo temporal de masterdatabase.inventory_metrics_current...
✅ Respaldo creado: masterdatabase.inventory_metrics_current_backup
📊 Filtrados 0 registros más recientes de IMC
✅ Tabla masterdatabase.inventory_metrics_current actualizada con solo registros más recientes.
🔍 Registros finales en IMC: 0


In [24]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("📊 Leyendo datos actuales de IMC...")
current_df = pd.read_sql("SELECT * FROM masterdatabase.inventory_metrics_current", engine)
print(f"📌 Total registros: {len(current_df)}")

# Análisis de duplicados
print("\n🔍 Analizando duplicados por contract_code...")
dup_analysis = (
    current_df.groupby("contract_code", dropna=False)["inventory_year"]
    .agg(list)
    .reset_index()
)
dup_analysis["count"] = dup_analysis["inventory_year"].apply(len)
contracts_with_multiple = dup_analysis[dup_analysis["count"] > 1]
print(f"   ├─ Contratos con múltiples registros: {len(contracts_with_multiple)}")
print(f"   └─ Contratos únicos: {len(dup_analysis)}")

# MÉTODO CORRECTO: Ordenar por inventory_year (NaN al final) y tomar el primero de cada grupo
print("\n🔍 Filtrando solo el registro más reciente por contract_code...")

# Primero eliminar duplicados exactos de contract_code + inventory_year
print("   ├─ Eliminando duplicados de contract_code + inventory_year...")
deduped = current_df.drop_duplicates(subset=['contract_code', 'inventory_year'], keep='first')
print(f"      └─ {len(current_df)} → {len(deduped)} registros")

# Ordenar y tomar el más reciente (NaN va al final automáticamente)
latest_df = (
    deduped
    .sort_values('inventory_year', ascending=False, na_position='last')
    .groupby('contract_code', dropna=False, as_index=False)
    .first()
)

print(f"   └─ Resultado: {len(latest_df)} registros (1 por contract)")

# Eliminar columna 'rn' si existe
if 'rn' in latest_df.columns:
    latest_df = latest_df.drop(columns=['rn'])
    print("\n🗑️ Columna 'rn' eliminada")

# Análisis del resultado
with_year = latest_df[latest_df['inventory_year'].notna()]
without_year = latest_df[latest_df['inventory_year'].isna()]
print(f"\n📊 Resultado del filtrado:")
print(f"   ├─ Con inventory_year: {len(with_year)}")
print(f"   └─ Sin inventory_year: {len(without_year)}")

# Mostrar preview
print("\n📋 Preview de registros a mantener (top 15 con año):")
preview = with_year[['contract_code', 'inventory_year', 'inventory_date', 'pkid']].head(15)
print(preview.to_string(index=False))

# CONFIRMACIÓN DE SEGURIDAD
print(f"\n⚠️  ATENCIÓN:")
print(f"   ├─ Total actual: {len(current_df)} registros")
print(f"   ├─ Se eliminarán: {len(current_df) - len(latest_df)} registros")
print(f"   └─ Se mantendrán: {len(latest_df)} registros (1 por contract)")

respuesta = input("\n¿Confirmas? Escribe 'SI' para continuar: ")

if respuesta.strip().upper() == 'SI':
    print("\n🚨 ACTUALIZANDO TABLA...")

    with engine.begin() as conn:
        conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics_current"))

    latest_df.to_sql(
        "inventory_metrics_current",
        con=engine,
        schema="masterdatabase",
        if_exists="append",
        index=False
    )

    print("✅ TABLA ACTUALIZADA EXITOSAMENTE")

    # Verificación final
    count = pd.read_sql("SELECT COUNT(*) as total FROM masterdatabase.inventory_metrics_current", engine)
    final = pd.read_sql(
        "SELECT contract_code, inventory_year, pkid FROM masterdatabase.inventory_metrics_current ORDER BY inventory_year DESC NULLS LAST LIMIT 10",
        engine
    )
    print(f"\n📊 Registros finales: {count.loc[0, 'total']}")
    print("\n✓ Top 10 por año más reciente:")
    print(final.to_string(index=False))
else:
    print("❌ Operación cancelada. No se hizo ningún cambio.")

💻 Conectado a la base de datos helloworldtree
📊 Leyendo datos actuales de IMC...
📌 Total registros: 521

🔍 Analizando duplicados por contract_code...
   ├─ Contratos con múltiples registros: 70
   └─ Contratos únicos: 424

🔍 Filtrando solo el registro más reciente por contract_code...
   ├─ Eliminando duplicados de contract_code + inventory_year...
      └─ 521 → 520 registros
   └─ Resultado: 424 registros (1 por contract)

🗑️ Columna 'rn' eliminada

📊 Resultado del filtrado:
   ├─ Con inventory_year: 198
   └─ Sin inventory_year: 226

📋 Preview de registros a mantener (top 15 con año):
contract_code  inventory_year inventory_date        pkid
       CR0013          2021.0     2021-01-01 CR0013 2021
       CR0028          2021.0     2021-01-01 CR0028 2021
       CR0029          2021.0     2021-01-01 CR0029 2021
       CR0030          2025.0     2025-01-23 CR0030 2025
       CR0035          2024.0     2024-02-09 CR0035 2024
       CR0039          2024.0     2024-01-16 CR0039 2024
      

In [27]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

# Leer el Excel
excel_path = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_metrics_current_latest.xlsx"

print("📂 Leyendo Excel...")
df = pd.read_excel(excel_path, sheet_name="Original_IMC")
print(f"📊 Registros en Excel: {len(df)}")

# Verificar columnas
print(f"🔍 Columnas: {list(df.columns)}")

# RESTAURAR
print("🚨 RESTAURANDO masterdatabase.inventory_metrics_current...")

with engine.begin() as conn:
    conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics_current"))

# Insertar TODO desde el Excel
df.to_sql(
    "inventory_metrics_current",
    con=engine,
    schema="masterdatabase",
    if_exists="append",
    index=False
)

print("✅ TABLA RESTAURADA DESDE EXCEL")

# Verificar
count = pd.read_sql("SELECT COUNT(*) as total FROM masterdatabase.inventory_metrics_current", engine)
print(f"📊 Registros restaurados: {count.loc[0, 'total']}")

💻 Conectado a la base de datos helloworldtree
📂 Leyendo Excel...
📊 Registros en Excel: 521
🔍 Columnas: ['rel_path', 'contract_code', 'planting_year', 'tree_age', 'inventory_year', 'inventory_date', 'survival', 'tht_mean', 'tht_std', 'mht_mean', 'mht_std', 'mht_pct_of_target', 'dbh_mean', 'dbh_std', 'noncom_dbh_count', 'dbh_pct_of_target', 'doyle_bf_mean', 'doyle_bf_std', 'doyle_bf_total', 'projected_dbh', 'projected_doyle_bf', 'pkid', 'progress', 'total_trees', 'mortality', 'planting_date', 'type_of_metric', 'contract_status', 'rn']
🚨 RESTAURANDO masterdatabase.inventory_metrics_current...


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "imc_unique_contract"
DETAIL:  Key (contract_code)=(CR0030) already exists.

[SQL: INSERT INTO masterdatabase.inventory_metrics_current (rel_path, contract_code, planting_year, tree_age, inventory_year, inventory_date, survival, tht_mean, tht_std, mht_mean, mht_std, mht_pct_of_target, dbh_mean, dbh_std, noncom_dbh_count, dbh_pct_of ... 330336 characters truncated ... lity__520)s, %(planting_date__520)s, %(type_of_metric__520)s, %(contract_status__520)s, %(rn__520)s)]
[parameters: {'planting_year__0': 2017.0, 'mht_pct_of_target__0': None, 'mortality__0': 0.1241, 'rel_path__0': None, 'mht_std__0': None, 'planting_date__0': datetime.datetime(2017, 7, 1, 0, 0), 'dbh_pct_of_target__0': None, 'tht_mean__0': 39.47, 'inventory_year__0': 2021.0, 'progress__0': 'OK', 'tree_age__0': '3 6/12', 'total_trees__0': 126.0, 'inventory_date__0': datetime.datetime(2021, 1, 1, 0, 0), 'type_of_metric__0': 'inven

In [28]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

# Leer el Excel
excel_path = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\inventory_metrics_current_latest.xlsx"

print("📂 Leyendo Excel...")
df = pd.read_excel(excel_path, sheet_name="Original_IMC")
print(f"📊 Registros en Excel: {len(df)}")
print(f"📊 Contratos únicos: {df['contract_code'].nunique()}")

# Eliminar duplicados - mantener solo el más reciente por contrato
print("\n🔍 Eliminando duplicados...")
df_sorted = df.sort_values(
    by=['contract_code', 'inventory_year', 'inventory_date', 'pkid'],
    ascending=[True, False, False, False],
    na_position='last'
)
df_unique = df_sorted.drop_duplicates(subset=['contract_code'], keep='first')

# Eliminar la columna 'rn' si existe (es solo para ordenamiento)
if 'rn' in df_unique.columns:
    df_unique = df_unique.drop(columns=['rn'])

print(f"📊 Después de eliminar duplicados: {len(df_unique)} registros")

# RESTAURAR IMC
print("\n🚨 RESTAURANDO masterdatabase.inventory_metrics_current...")

with engine.begin() as conn:
    # Eliminar constraint temporalmente
    print("🔓 Eliminando constraint temporalmente...")
    conn.execute(text("""
        ALTER TABLE masterdatabase.inventory_metrics_current
        DROP CONSTRAINT IF EXISTS imc_unique_contract
    """))

    print("🗑️ Vaciando tabla...")
    conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics_current CASCADE"))

# Insertar datos únicos
print("📥 Insertando datos únicos...")
df_unique.to_sql(
    "inventory_metrics_current",
    con=engine,
    schema="masterdatabase",
    if_exists="append",
    index=False
)

# Restaurar constraint
print("🔒 Restaurando constraint...")
with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE masterdatabase.inventory_metrics_current
        ADD CONSTRAINT imc_unique_contract UNIQUE (contract_code)
    """))

print("\n✅ TABLA RESTAURADA DESDE EXCEL")

# Verificar
count = pd.read_sql("SELECT COUNT(*) as total FROM masterdatabase.inventory_metrics_current", engine)
print(f"📊 Registros restaurados: {count.loc[0, 'total']}")

# Verificar no hay duplicados
duplicates = pd.read_sql("""
    SELECT contract_code, COUNT(*) as count
    FROM masterdatabase.inventory_metrics_current
    GROUP BY contract_code
    HAVING COUNT(*) > 1
""", engine)
print(f"📊 Contratos duplicados: {len(duplicates)}")

💻 Conectado a la base de datos helloworldtree
📂 Leyendo Excel...
📊 Registros en Excel: 521
📊 Contratos únicos: 424

🔍 Eliminando duplicados...
📊 Después de eliminar duplicados: 424 registros

🚨 RESTAURANDO masterdatabase.inventory_metrics_current...
🔓 Eliminando constraint temporalmente...
🗑️ Vaciando tabla...
📥 Insertando datos únicos...
🔒 Restaurando constraint...

✅ TABLA RESTAURADA DESDE EXCEL
📊 Registros restaurados: 424
📊 Contratos duplicados: 0


In [35]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*60)
print("🗑️ ELIMINANDO TRIGGER VIEJO")
print("="*60)

with engine.begin() as conn:
    print("🔧 Eliminando trigger sync_inventory_to_current...")
    conn.execute(text("""
        DROP TRIGGER IF EXISTS trigger_sync_inventory_to_current
        ON masterdatabase.inventory_metrics CASCADE
    """))

    print("🔧 Eliminando función sync_inventory_to_current...")
    conn.execute(text("""
        DROP FUNCTION IF EXISTS masterdatabase.sync_inventory_to_current() CASCADE
    """))

print("✅ Trigger y función eliminados")

print("\n" + "="*60)
print("🔄 COPIANDO IMC → IM (SIN TRIGGER)")
print("="*60)

# Obtener columnas de IM (sin las que están solo en IMC)
common_cols = pd.read_sql("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'masterdatabase'
      AND table_name = 'inventory_metrics'
    ORDER BY ordinal_position
""", engine)['column_name'].tolist()

cols_str = ', '.join(common_cols)

with engine.begin() as conn:
    print("\n🗑️ Vaciando inventory_metrics...")
    conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics CASCADE"))

    print(f"📥 Copiando {len(common_cols)} columnas de IMC → IM...")

    copy_sql = f"""
        INSERT INTO masterdatabase.inventory_metrics ({cols_str})
        SELECT {cols_str}
        FROM masterdatabase.inventory_metrics_current
    """

    conn.execute(text(copy_sql))

    im_count = conn.execute(text("""
        SELECT COUNT(*) FROM masterdatabase.inventory_metrics
    """)).scalar()
    print(f"   ✅ {im_count} registros copiados")

# Verificar
print("\n📊 VERIFICACIÓN...")
stats = pd.read_sql("""
    SELECT
        'inventory_metrics' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos
    FROM masterdatabase.inventory_metrics
    UNION ALL
    SELECT
        'inventory_metrics_current' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos
    FROM masterdatabase.inventory_metrics_current
""", engine)
print(stats.to_string(index=False))

print("\n✅ Copia completada exitosamente")

💻 Conectado a la base de datos helloworldtree
🗑️ ELIMINANDO TRIGGER VIEJO
🔧 Eliminando trigger sync_inventory_to_current...
🔧 Eliminando función sync_inventory_to_current...
✅ Trigger y función eliminados

🔄 COPIANDO IMC → IM (SIN TRIGGER)

🗑️ Vaciando inventory_metrics...
📥 Copiando 27 columnas de IMC → IM...
   ✅ 424 registros copiados

📊 VERIFICACIÓN...
                    tabla  registros  contratos_unicos
        inventory_metrics        424               424
inventory_metrics_current        424               424

✅ Copia completada exitosamente


In [36]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*60)
print("🔧 ARMONIZANDO TIPOS DE DATOS")
print("="*60)

# Ver tipos conflictivos
type_comparison = pd.read_sql("""
    SELECT
        im.column_name,
        im.data_type as im_type,
        imc.data_type as imc_type
    FROM information_schema.columns im
    JOIN information_schema.columns imc
        ON im.column_name = imc.column_name
    WHERE im.table_schema = 'masterdatabase'
      AND im.table_name = 'inventory_metrics'
      AND imc.table_schema = 'masterdatabase'
      AND imc.table_name = 'inventory_metrics_current'
      AND im.data_type != imc.data_type
    ORDER BY im.column_name
""", engine)

print(f"\n❌ Columnas con tipos diferentes: {len(type_comparison)}")
if len(type_comparison) > 0:
    print(type_comparison.to_string(index=False))

# Decisión: Como IM es la tabla histórica "oficial" y queremos que IMC
# tenga la misma estructura, vamos a cambiar IMC para que coincida con IM

print("\n🔧 Ajustando tipos en inventory_metrics_current para que coincidan con inventory_metrics...")

with engine.begin() as conn:
    # Cambiar planting_year de text a double precision en IMC
    print("   📝 Cambiando planting_year: text → double precision")
    conn.execute(text("""
        ALTER TABLE masterdatabase.inventory_metrics_current
        ALTER COLUMN planting_year TYPE double precision
        USING planting_year::double precision
    """))

    # Si hay otros tipos conflictivos, agregarlos aquí
    # (basándonos en el output del query anterior)

print("✅ Tipos armonizados")

# Verificar que ya no hay conflictos
print("\n🔍 Verificando...")
type_check = pd.read_sql("""
    SELECT
        im.column_name,
        im.data_type as im_type,
        imc.data_type as imc_type
    FROM information_schema.columns im
    JOIN information_schema.columns imc
        ON im.column_name = imc.column_name
    WHERE im.table_schema = 'masterdatabase'
      AND im.table_name = 'inventory_metrics'
      AND imc.table_schema = 'masterdatabase'
      AND imc.table_name = 'inventory_metrics_current'
      AND im.data_type != imc.data_type
""", engine)

if len(type_check) == 0:
    print("✅ Todos los tipos coinciden ahora")
else:
    print("⚠️ Aún hay tipos diferentes:")
    print(type_check.to_string(index=False))

print("\n" + "="*60)
print("🔄 COPIANDO IMC → IM")
print("="*60)

# Obtener columnas comunes (sin rn y contract_status que no están en IM)
common_cols = pd.read_sql("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'masterdatabase'
      AND table_name = 'inventory_metrics'
    ORDER BY ordinal_position
""", engine)['column_name'].tolist()

cols_str = ', '.join(common_cols)

with engine.begin() as conn:
    print("\n🗑️ Vaciando inventory_metrics...")
    conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics CASCADE"))

    print(f"📥 Copiando {len(common_cols)} columnas de IMC → IM...")

    copy_sql = f"""
        INSERT INTO masterdatabase.inventory_metrics ({cols_str})
        SELECT {cols_str}
        FROM masterdatabase.inventory_metrics_current
    """

    conn.execute(text(copy_sql))

    im_count = conn.execute(text("""
        SELECT COUNT(*) FROM masterdatabase.inventory_metrics
    """)).scalar()
    print(f"   ✅ {im_count} registros copiados")

# Verificar
print("\n📊 VERIFICACIÓN...")
stats = pd.read_sql("""
    SELECT
        'inventory_metrics' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos
    FROM masterdatabase.inventory_metrics
    UNION ALL
    SELECT
        'inventory_metrics_current' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos
    FROM masterdatabase.inventory_metrics_current
""", engine)
print(stats.to_string(index=False))

print("\n✅ Copia completada exitosamente")
print("📝 Siguiente paso: Recrear trigger de sincronización")

💻 Conectado a la base de datos helloworldtree
🔧 ARMONIZANDO TIPOS DE DATOS

❌ Columnas con tipos diferentes: 0

🔧 Ajustando tipos en inventory_metrics_current para que coincidan con inventory_metrics...
   📝 Cambiando planting_year: text → double precision
✅ Tipos armonizados

🔍 Verificando...
✅ Todos los tipos coinciden ahora

🔄 COPIANDO IMC → IM

🗑️ Vaciando inventory_metrics...
📥 Copiando 27 columnas de IMC → IM...
   ✅ 424 registros copiados

📊 VERIFICACIÓN...
                    tabla  registros  contratos_unicos
        inventory_metrics        424               424
inventory_metrics_current        424               424

✅ Copia completada exitosamente
📝 Siguiente paso: Recrear trigger de sincronización


In [32]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*60)
print("🔍 DIAGNOSTICANDO DIFERENCIAS DE ESTRUCTURA")
print("="*60)

# Ver columnas de cada tabla
im_cols = pd.read_sql("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'masterdatabase'
      AND table_name = 'inventory_metrics'
    ORDER BY ordinal_position
""", engine)

imc_cols = pd.read_sql("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'masterdatabase'
      AND table_name = 'inventory_metrics_current'
    ORDER BY ordinal_position
""", engine)

print(f"\n📊 inventory_metrics tiene {len(im_cols)} columnas")
print(f"📊 inventory_metrics_current tiene {len(imc_cols)} columnas")

# Encontrar columnas comunes
common_cols = set(im_cols['column_name']) & set(imc_cols['column_name'])
only_in_im = set(im_cols['column_name']) - set(imc_cols['column_name'])
only_in_imc = set(imc_cols['column_name']) - set(im_cols['column_name'])

print(f"\n✅ Columnas comunes: {len(common_cols)}")
if only_in_im:
    print(f"⚠️ Solo en IM: {', '.join(only_in_im)}")
if only_in_imc:
    print(f"⚠️ Solo en IMC: {', '.join(only_in_imc)}")

# Generar lista de columnas comunes ordenadas
common_cols_ordered = [col for col in im_cols['column_name'] if col in common_cols]
cols_str = ', '.join(common_cols_ordered)

print(f"\n🔧 Usaremos estas {len(common_cols_ordered)} columnas comunes para la copia")

print("\n" + "="*60)
print("🔄 MIGRACIÓN CORREGIDA")
print("="*60)

with engine.begin() as conn:
    # Vaciar IM
    print("\n🗑️ Vaciando inventory_metrics...")
    conn.execute(text("TRUNCATE TABLE masterdatabase.inventory_metrics CASCADE"))

    # Copiar solo columnas comunes
    print(f"📥 Copiando {len(common_cols_ordered)} columnas de IMC → IM...")

    copy_sql = f"""
        INSERT INTO masterdatabase.inventory_metrics ({cols_str})
        SELECT {cols_str}
        FROM masterdatabase.inventory_metrics_current
    """

    conn.execute(text(copy_sql))

    im_count = conn.execute(text("""
        SELECT COUNT(*) FROM masterdatabase.inventory_metrics
    """)).scalar()
    print(f"   ✅ {im_count} registros copiados a inventory_metrics")

# Verificar
print("\n📊 VERIFICACIÓN...")
stats = pd.read_sql("""
    SELECT
        'inventory_metrics (histórica)' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos
    FROM masterdatabase.inventory_metrics
    UNION ALL
    SELECT
        'inventory_metrics_current' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos
    FROM masterdatabase.inventory_metrics_current
""", engine)
print(stats.to_string(index=False))

print("\n✅ Copia completada. Ahora puedes ejecutar la parte 3️⃣ del script anterior (recrear trigger)")

💻 Conectado a la base de datos helloworldtree
🔍 DIAGNOSTICANDO DIFERENCIAS DE ESTRUCTURA

📊 inventory_metrics tiene 27 columnas
📊 inventory_metrics_current tiene 29 columnas

✅ Columnas comunes: 27
⚠️ Solo en IMC: rn, contract_status

🔧 Usaremos estas 27 columnas comunes para la copia

🔄 MIGRACIÓN CORREGIDA

🗑️ Vaciando inventory_metrics...
📥 Copiando 27 columnas de IMC → IM...


ProgrammingError: (psycopg2.errors.DatatypeMismatch) column "planting_year" is of type double precision but expression is of type text
LINE 3: ..._trees, mortality, type_of_metric, planting_date, planting_y...
                                                             ^
HINT:  You will need to rewrite or cast the expression.

[SQL: 
        INSERT INTO masterdatabase.inventory_metrics (rel_path, contract_code, tree_age, inventory_year, inventory_date, survival, tht_mean, tht_std, mht_mean, mht_std, mht_pct_of_target, dbh_mean, dbh_std, noncom_dbh_count, dbh_pct_of_target, doyle_bf_mean, doyle_bf_std, doyle_bf_total, projected_dbh, projected_doyle_bf, pkid, progress, total_trees, mortality, type_of_metric, planting_date, planting_year)
        SELECT rel_path, contract_code, tree_age, inventory_year, inventory_date, survival, tht_mean, tht_std, mht_mean, mht_std, mht_pct_of_target, dbh_mean, dbh_std, noncom_dbh_count, dbh_pct_of_target, doyle_bf_mean, doyle_bf_std, doyle_bf_total, pro

In [37]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*60)
print("⚙️ RECREANDO TRIGGER DE SINCRONIZACIÓN IM → IMC")
print("="*60)

with engine.begin() as conn:
    print("\n🔧 Creando función sync_inventory_to_current()...")
    conn.execute(text("""
        CREATE OR REPLACE FUNCTION masterdatabase.sync_inventory_to_current()
        RETURNS TRIGGER AS $$
        DECLARE
            latest_year integer;
            latest_date date;
            latest_pkid text;
        BEGIN
            -- Obtener el registro más reciente en IMC para este contrato
            SELECT inventory_year, inventory_date, pkid
            INTO latest_year, latest_date, latest_pkid
            FROM masterdatabase.inventory_metrics_current
            WHERE contract_code = NEW.contract_code
            LIMIT 1;

            -- Si no existe en IMC, insertamos
            IF latest_year IS NULL THEN
                INSERT INTO masterdatabase.inventory_metrics_current (
                    rel_path, contract_code, tree_age, inventory_year,
                    inventory_date, survival, tht_mean, tht_std, mht_mean,
                    mht_std, mht_pct_of_target, dbh_mean, dbh_std,
                    noncom_dbh_count, dbh_pct_of_target, doyle_bf_mean,
                    doyle_bf_std, doyle_bf_total, projected_dbh,
                    projected_doyle_bf, pkid, progress, total_trees,
                    mortality, type_of_metric, planting_date, planting_year
                )
                SELECT
                    rel_path, contract_code, tree_age, inventory_year,
                    inventory_date, survival, tht_mean, tht_std, mht_mean,
                    mht_std, mht_pct_of_target, dbh_mean, dbh_std,
                    noncom_dbh_count, dbh_pct_of_target, doyle_bf_mean,
                    doyle_bf_std, doyle_bf_total, projected_dbh,
                    projected_doyle_bf, pkid, progress, total_trees,
                    mortality, type_of_metric, planting_date, planting_year
                FROM masterdatabase.inventory_metrics
                WHERE pkid = NEW.pkid
                ON CONFLICT (contract_code) DO NOTHING;

                RAISE NOTICE '🆕 [IMC SYNC] Nuevo contrato % agregado (Year: %, Date: %)',
                             NEW.contract_code, NEW.inventory_year, NEW.inventory_date;
                RETURN NEW;
            END IF;

            -- Si el nuevo registro es más reciente, actualizamos
            IF (NEW.inventory_year > latest_year)
               OR (NEW.inventory_year = latest_year
                   AND COALESCE(NEW.inventory_date, '1900-01-01'::date) > COALESCE(latest_date, '1900-01-01'::date))
            THEN
                UPDATE masterdatabase.inventory_metrics_current
                SET
                    rel_path = NEW.rel_path,
                    tree_age = NEW.tree_age,
                    inventory_year = NEW.inventory_year,
                    inventory_date = NEW.inventory_date,
                    survival = NEW.survival,
                    tht_mean = NEW.tht_mean,
                    tht_std = NEW.tht_std,
                    mht_mean = NEW.mht_mean,
                    mht_std = NEW.mht_std,
                    mht_pct_of_target = NEW.mht_pct_of_target,
                    dbh_mean = NEW.dbh_mean,
                    dbh_std = NEW.dbh_std,
                    noncom_dbh_count = NEW.noncom_dbh_count,
                    dbh_pct_of_target = NEW.dbh_pct_of_target,
                    doyle_bf_mean = NEW.doyle_bf_mean,
                    doyle_bf_std = NEW.doyle_bf_std,
                    doyle_bf_total = NEW.doyle_bf_total,
                    projected_dbh = NEW.projected_dbh,
                    projected_doyle_bf = NEW.projected_doyle_bf,
                    pkid = NEW.pkid,
                    progress = NEW.progress,
                    total_trees = NEW.total_trees,
                    mortality = NEW.mortality,
                    type_of_metric = NEW.type_of_metric,
                    planting_date = NEW.planting_date,
                    planting_year = NEW.planting_year
                WHERE contract_code = NEW.contract_code;

                RAISE NOTICE '♻️ [IMC SYNC] Contrato % actualizado | Anterior: %/% → Nuevo: %/%',
                             NEW.contract_code,
                             latest_year, latest_date,
                             NEW.inventory_year, NEW.inventory_date;
            ELSE
                RAISE NOTICE '⏩ [IMC SYNC] Inventario ignorado para %: %/% no es más reciente que %/%',
                             NEW.contract_code,
                             NEW.inventory_year, NEW.inventory_date,
                             latest_year, latest_date;
            END IF;

            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    """))

    print("🔧 Creando trigger...")
    conn.execute(text("""
        CREATE TRIGGER trigger_sync_inventory_to_current
        AFTER INSERT OR UPDATE ON masterdatabase.inventory_metrics
        FOR EACH ROW
        EXECUTE FUNCTION masterdatabase.sync_inventory_to_current()
    """))

print("✅ Función y trigger creados correctamente")

# ============================================================
# VERIFICACIÓN
# ============================================================
print("\n" + "="*60)
print("✅ VERIFICACIÓN FINAL")
print("="*60)

# Verificar que el trigger está instalado
trigger_check = pd.read_sql("""
    SELECT
        trigger_name,
        event_manipulation as event,
        event_object_table as table_name,
        action_statement
    FROM information_schema.triggers
    WHERE trigger_schema = 'masterdatabase'
      AND event_object_table = 'inventory_metrics'
      AND trigger_name = 'trigger_sync_inventory_to_current'
""", engine)

print("\n📋 Trigger instalado:")
if len(trigger_check) > 0:
    for _, row in trigger_check.iterrows():
        print(f"   ✅ {row['trigger_name']} - Evento: {row['event']}")
else:
    print("   ❌ Trigger no encontrado")

# Verificar estado de las tablas
stats = pd.read_sql("""
    SELECT
        'inventory_metrics (histórica)' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos,
        MIN(inventory_year) as year_min,
        MAX(inventory_year) as year_max
    FROM masterdatabase.inventory_metrics
    UNION ALL
    SELECT
        'inventory_metrics_current' as tabla,
        COUNT(*) as registros,
        COUNT(DISTINCT contract_code) as contratos_unicos,
        MIN(inventory_year) as year_min,
        MAX(inventory_year) as year_max
    FROM masterdatabase.inventory_metrics_current
""", engine)

print("\n📊 Estado de las tablas:")
print(stats.to_string(index=False))

# Verificar que ambas tablas tienen los mismos contratos
sync_check = pd.read_sql("""
    WITH im_contracts AS (
        SELECT DISTINCT contract_code FROM masterdatabase.inventory_metrics
    ),
    imc_contracts AS (
        SELECT DISTINCT contract_code FROM masterdatabase.inventory_metrics_current
    )
    SELECT
        COUNT(*) as contratos_solo_en_im
    FROM im_contracts
    LEFT JOIN imc_contracts USING (contract_code)
    WHERE imc_contracts.contract_code IS NULL
""", engine)

solo_en_im = sync_check.loc[0, 'contratos_solo_en_im']
if solo_en_im == 0:
    print("\n✅ Todos los contratos de IM están en IMC")
else:
    print(f"\n⚠️ Hay {solo_en_im} contratos en IM que no están en IMC")

print("\n" + "="*60)
print("🎉 MIGRACIÓN COMPLETADA EXITOSAMENTE")
print("="*60)

print("\n✅ RESUMEN:")
print("   • inventory_metrics = Tabla histórica completa (424 registros)")
print("   • inventory_metrics_current = Registros más recientes por contrato (424 registros)")
print("   • Trigger activo: IM → IMC automático")
print("   • Tipos de datos armonizados (planting_year = double precision)")
print("   • Constraint de unicidad en IMC por contract_code")

print("\n💡 PRÓXIMOS PASOS:")
print("   1. ✅ Convertir survival_current en VIEW sobre IMC")
print("   2. ✅ Eliminar/convertir survival_timeseries")
print("   3. ✅ Actualizar reportes para usar las nuevas estructuras")
print("   4. ✅ Eliminar trigger CTI → IMC (ya no es necesario)")

💻 Conectado a la base de datos helloworldtree
⚙️ RECREANDO TRIGGER DE SINCRONIZACIÓN IM → IMC

🔧 Creando función sync_inventory_to_current()...
🔧 Creando trigger...
✅ Función y trigger creados correctamente

✅ VERIFICACIÓN FINAL

📋 Trigger instalado:
   ✅ trigger_sync_inventory_to_current - Evento: INSERT
   ✅ trigger_sync_inventory_to_current - Evento: UPDATE

📊 Estado de las tablas:
                        tabla  registros  contratos_unicos  year_min  year_max
inventory_metrics (histórica)        424               424      2021      2025
    inventory_metrics_current        424               424      2021      2025

✅ Todos los contratos de IM están en IMC

🎉 MIGRACIÓN COMPLETADA EXITOSAMENTE

✅ RESUMEN:
   • inventory_metrics = Tabla histórica completa (424 registros)
   • inventory_metrics_current = Registros más recientes por contrato (424 registros)
   • Trigger activo: IM → IMC automático
   • Tipos de datos armonizados (planting_year = double precision)
   • Constraint de unici

In [39]:
# ============================================================
# 5️⃣ AUDITORÍA FINAL
# ============================================================
print("\n5️⃣ AUDITORÍA FINAL...")

# Verificar que IM e IMC tienen los mismos contratos
audit = pd.read_sql("""
    WITH im_contracts AS (
        SELECT DISTINCT contract_code FROM masterdatabase.inventory_metrics
    ),
    imc_contracts AS (
        SELECT DISTINCT contract_code FROM masterdatabase.inventory_metrics_current
    )
    SELECT
        COALESCE(im.contract_code, imc.contract_code) as contract_code,
        CASE
            WHEN im.contract_code IS NULL THEN 'Solo en IMC'
            WHEN imc.contract_code IS NULL THEN 'Solo en IM'
            ELSE 'En ambas'
        END as status
    FROM im_contracts im
    FULL OUTER JOIN imc_contracts imc ON im.contract_code = imc.contract_code
    WHERE im.contract_code IS NULL OR imc.contract_code IS NULL
""", engine)

if len(audit) == 0:
    print("   ✅ Todos los contratos están sincronizados entre IM e IMC")
else:
    print("   ⚠️ Contratos con discrepancias:")
    print(audit.to_string(index=False))

# Verificar duplicados en IM (debería permitirlos - es histórica)
duplicates_im = pd.read_sql("""
    SELECT contract_code, COUNT(*) as count
    FROM masterdatabase.inventory_metrics
    GROUP BY contract_code
    HAVING COUNT(*) > 1
    ORDER BY count DESC
    LIMIT 5
""", engine)

if len(duplicates_im) > 0:
    print(f"\n   ℹ️ IM tiene contratos con múltiples inventarios (normal en tabla histórica):")
    print(f"   Ejemplo: {duplicates_im.iloc[0]['contract_code']} tiene {duplicates_im.iloc[0]['count']} registros")
else:
    print("\n   ℹ️ IM tiene 1 registro por contrato (por ahora)")

# Verificar duplicados en IMC (NO debe tenerlos)
duplicates_imc = pd.read_sql("""
    SELECT contract_code, COUNT(*) as count
    FROM masterdatabase.inventory_metrics_current
    GROUP BY contract_code
    HAVING COUNT(*) > 1
""", engine)

if len(duplicates_imc) == 0:
    print("   ✅ IMC no tiene duplicados (correcto)")
else:
    print("   ❌ IMC tiene duplicados (ERROR):")
    print(duplicates_imc.to_string(index=False))

print("\n" + "="*60)
print("🎉 MIGRACIÓN COMPLETADA EXITOSAMENTE")
print("="*60)
print("\n📊 ESTADO FINAL:")
print("   ✅ inventory_metrics = Tabla histórica completa")
print("   ✅ inventory_metrics_current = Registros más recientes por contrato")
print("   ✅ Trigger activo: IM → IMC automático")
print("   ✅ Constraint de unicidad en IMC")
print("\n💡 PRÓXIMOS PASOS:")
print("   1. Convertir survival_current en VIEW sobre IMC")
print("   2. Eliminar/convertir survival_timeseries")
print("   3. Actualizar reportes para usar nuevas estructuras")


5️⃣ AUDITORÍA FINAL...
   ✅ Todos los contratos están sincronizados entre IM e IMC

   ℹ️ IM tiene 1 registro por contrato (por ahora)
   ✅ IMC no tiene duplicados (correcto)

🎉 MIGRACIÓN COMPLETADA EXITOSAMENTE

📊 ESTADO FINAL:
   ✅ inventory_metrics = Tabla histórica completa
   ✅ inventory_metrics_current = Registros más recientes por contrato
   ✅ Trigger activo: IM → IMC automático
   ✅ Constraint de unicidad en IMC

💡 PRÓXIMOS PASOS:
   1. Convertir survival_current en VIEW sobre IMC
   2. Eliminar/convertir survival_timeseries
   3. Actualizar reportes para usar nuevas estructuras


In [40]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*60)
print("🔄 CONVIRTIENDO survival_current EN VIEW")
print("="*60)

with engine.begin() as conn:
    # 1. Verificar qué tipo de objeto es survival_current actualmente
    print("\n🔍 Verificando tipo actual de survival_current...")
    current_type = pd.read_sql("""
        SELECT table_type
        FROM information_schema.tables
        WHERE table_schema = 'masterdatabase'
          AND table_name = 'survival_current'
    """, engine)

    if len(current_type) > 0:
        print(f"   Tipo actual: {current_type.loc[0, 'table_type']}")
    else:
        print("   ⚠️ survival_current no existe")

    # 2. Respaldar si es tabla
    if len(current_type) > 0 and current_type.loc[0, 'table_type'] == 'BASE TABLE':
        print("\n📦 Respaldando survival_current actual...")
        conn.execute(text("""
            DROP TABLE IF EXISTS masterdatabase.survival_current_backup CASCADE
        """))
        conn.execute(text("""
            CREATE TABLE masterdatabase.survival_current_backup AS
            SELECT *, NOW() as backup_date
            FROM masterdatabase.survival_current
        """))

        backup_count = conn.execute(text("""
            SELECT COUNT(*) FROM masterdatabase.survival_current_backup
        """)).scalar()
        print(f"   ✅ {backup_count} registros respaldados")

    # 3. Eliminar objeto actual (tabla o vista)
    print("\n🗑️ Eliminando survival_current actual...")
    conn.execute(text("""
        DROP TABLE IF EXISTS masterdatabase.survival_current CASCADE
    """))
    conn.execute(text("""
        DROP VIEW IF EXISTS masterdatabase.survival_current CASCADE
    """))
    conn.execute(text("""
        DROP MATERIALIZED VIEW IF EXISTS masterdatabase.survival_current CASCADE
    """))

    # 4. Crear como VIEW que lee de IMC
    print("🏗️ Creando VIEW survival_current sobre IMC...")
    conn.execute(text("""
        CREATE OR REPLACE VIEW masterdatabase.survival_current AS
        SELECT
            contract_code,
            survival AS current_survival_pct,
            total_trees AS current_surviving_trees,
            CASE
                WHEN type_of_metric = 'placeholder' THEN 'Placeholder (IMC)'
                WHEN inventory_year IS NOT NULL THEN 'Inventory ' || inventory_year::text
                ELSE 'IMC (unspecified)'
            END AS survival_metric_source,
            inventory_year,
            inventory_date,
            mortality
        FROM masterdatabase.inventory_metrics_current
        WHERE survival IS NOT NULL;
    """))

print("✅ survival_current convertida en VIEW")

# ============================================================
# VERIFICACIÓN
# ============================================================
print("\n" + "="*60)
print("🔍 VERIFICACIÓN")
print("="*60)

# Verificar que es una VIEW
view_check = pd.read_sql("""
    SELECT
        table_name,
        table_type
    FROM information_schema.tables
    WHERE table_schema = 'masterdatabase'
      AND table_name = 'survival_current'
""", engine)

if len(view_check) > 0:
    print(f"\n✅ survival_current es ahora: {view_check.loc[0, 'table_type']}")
else:
    print("\n❌ survival_current no existe")

# Contar registros
survival_stats = pd.read_sql("""
    SELECT
        COUNT(*) as total_registros,
        COUNT(DISTINCT contract_code) as contratos_unicos,
        COUNT(CASE WHEN current_survival_pct IS NOT NULL THEN 1 END) as con_survival,
        MIN(inventory_year) as year_min,
        MAX(inventory_year) as year_max
    FROM masterdatabase.survival_current
""", engine)

print(f"\n📊 Estadísticas de survival_current:")
print(f"   Total registros: {survival_stats.loc[0, 'total_registros']}")
print(f"   Contratos únicos: {survival_stats.loc[0, 'contratos_unicos']}")
print(f"   Con survival: {survival_stats.loc[0, 'con_survival']}")
print(f"   Años: {survival_stats.loc[0, 'year_min']} - {survival_stats.loc[0, 'year_max']}")

# Preview
print("\n📋 Preview de survival_current (top 5):")
preview = pd.read_sql("""
    SELECT
        contract_code,
        current_survival_pct,
        current_surviving_trees,
        survival_metric_source,
        inventory_year
    FROM masterdatabase.survival_current
    ORDER BY inventory_year DESC NULLS LAST
    LIMIT 5
""", engine)
print(preview.to_string(index=False))

print("\n✅ Conversión completada exitosamente")

💻 Conectado a la base de datos helloworldtree
🔄 CONVIRTIENDO survival_current EN VIEW

🔍 Verificando tipo actual de survival_current...
   Tipo actual: BASE TABLE

📦 Respaldando survival_current actual...
   ✅ 428 registros respaldados

🗑️ Eliminando survival_current actual...
🏗️ Creando VIEW survival_current sobre IMC...
✅ survival_current convertida en VIEW

🔍 VERIFICACIÓN

✅ survival_current es ahora: VIEW

📊 Estadísticas de survival_current:
   Total registros: 198
   Contratos únicos: 198
   Con survival: 198
   Años: 2021 - 2025

📋 Preview de survival_current (top 5):
contract_code current_survival_pct  current_surviving_trees survival_metric_source  inventory_year
       CR0074               0.7089                    112.0         Inventory 2025            2025
       CR0075               0.7857                     76.0         Inventory 2025            2025
       CR0030               0.6234                     48.0         Inventory 2025            2025
       CR0073          

In [42]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*60)
print("🧹 LIMPIANDO TRIGGERS OBSOLETOS")
print("="*60)

# Listar todos los triggers
all_triggers = pd.read_sql("""
    SELECT
        event_object_table as tabla,
        trigger_name,
        event_manipulation as evento
    FROM information_schema.triggers
    WHERE trigger_schema = 'masterdatabase'
    ORDER BY event_object_table, trigger_name
""", engine)

print("\n📋 Triggers actuales en masterdatabase:")
if len(all_triggers) > 0:
    print(all_triggers.to_string(index=False))
else:
    print("   ℹ️ No hay triggers")

# Buscar triggers en CTI
cti_triggers = all_triggers[all_triggers['tabla'] == 'contract_tree_information']

if len(cti_triggers) > 0:
    print(f"\n⚠️ Se encontraron {len(cti_triggers)} trigger(s) en contract_tree_information:")
    print(cti_triggers.to_string(index=False))

    print("\n💡 Arquitectura actual:")
    print("   CTI (manual) → IM (trigger) → IMC")
    print("\n💡 Los triggers CTI → IMC directos ya no son necesarios")

    with engine.begin() as conn:
        print("\n🗑️ Eliminando triggers obsoletos de CTI...")
        for trigger_name in cti_triggers['trigger_name']:
            # Eliminar trigger
            conn.execute(text(f"""
                DROP TRIGGER IF EXISTS {trigger_name}
                ON masterdatabase.contract_tree_information CASCADE
            """))
            print(f"   ✅ Trigger eliminado: {trigger_name}")

            # Intentar eliminar función asociada
            func_name = trigger_name.replace('trigger_', '')
            try:
                conn.execute(text(f"""
                    DROP FUNCTION IF EXISTS masterdatabase.{func_name}() CASCADE
                """))
                print(f"   ✅ Función eliminada: {func_name}()")
            except:
                pass
else:
    print("\n✅ No hay triggers en contract_tree_information (perfecto)")

# Verificar triggers finales
print("\n" + "="*60)
print("✅ VERIFICACIÓN FINAL DE TRIGGERS")
print("="*60)

final_triggers = pd.read_sql("""
    SELECT
        event_object_table as tabla,
        trigger_name,
        event_manipulation as evento
    FROM information_schema.triggers
    WHERE trigger_schema = 'masterdatabase'
    ORDER BY event_object_table, trigger_name
""", engine)

print("\n📋 Triggers que quedan (deberían ser solo IM → IMC):")
if len(final_triggers) > 0:
    print(final_triggers.to_string(index=False))
else:
    print("   ⚠️ No hay triggers (debería haber IM → IMC)")

print("\n✅ Limpieza completada")

💻 Conectado a la base de datos helloworldtree
🧹 LIMPIANDO TRIGGERS OBSOLETOS

📋 Triggers actuales en masterdatabase:
                    tabla                      trigger_name evento
contract_tree_information                     trg_cti_to_ca INSERT
contract_tree_information               trigger_sync_to_imc INSERT
        inventory_metrics trigger_sync_inventory_to_current INSERT
        inventory_metrics trigger_sync_inventory_to_current UPDATE

⚠️ Se encontraron 2 trigger(s) en contract_tree_information:
                    tabla        trigger_name evento
contract_tree_information       trg_cti_to_ca INSERT
contract_tree_information trigger_sync_to_imc INSERT

💡 Arquitectura actual:
   CTI (manual) → IM (trigger) → IMC

💡 Los triggers CTI → IMC directos ya no son necesarios

🗑️ Eliminando triggers obsoletos de CTI...
   ✅ Trigger eliminado: trg_cti_to_ca
   ✅ Función eliminada: trg_cti_to_ca()
   ✅ Trigger eliminado: trigger_sync_to_imc
   ✅ Función eliminada: sync_to_imc()

✅ VER

In [43]:
print("\n" + "="*60)
print("🧹 LIMPIANDO TRIGGERS OBSOLETOS")
print("="*60)

# Listar todos los triggers actuales
all_triggers = pd.read_sql("""
    SELECT
        event_object_table as tabla,
        trigger_name,
        event_manipulation as evento,
        action_statement
    FROM information_schema.triggers
    WHERE trigger_schema = 'masterdatabase'
    ORDER BY event_object_table, trigger_name
""", engine)

print("\n📋 Triggers actuales en masterdatabase:")
if len(all_triggers) > 0:
    print(all_triggers[['tabla', 'trigger_name', 'evento']].to_string(index=False))
else:
    print("   ℹ️ No hay triggers")

# Buscar específicamente triggers en CTI
cti_triggers = all_triggers[all_triggers['tabla'] == 'contract_tree_information']

if len(cti_triggers) > 0:
    print(f"\n⚠️ Se encontraron {len(cti_triggers)} trigger(s) en contract_tree_information:")
    print(cti_triggers[['trigger_name', 'evento']].to_string(index=False))

    # Preguntar si eliminar
    print("\n💡 Estos triggers podrían estar sincronizando CTI → IMC (ya no necesario)")
    print("   Ahora usamos: CTI → IM → IMC (con trigger automático)")

    # Mostrar detalles de cada trigger
    for _, trigger in cti_triggers.iterrows():
        print(f"\n📝 Trigger: {trigger['trigger_name']}")
        # Intentar extraer info del action_statement
        if 'inventory_metrics_current' in trigger['action_statement']:
            print("   ⚠️ Este trigger menciona inventory_metrics_current")
            print("   🗑️ Debería eliminarse (arquitectura obsoleta)")

    with engine.begin() as conn:
        print("\n🗑️ Eliminando triggers obsoletos de CTI...")
        for trigger_name in cti_triggers['trigger_name']:
            conn.execute(text(f"""
                DROP TRIGGER IF EXISTS {trigger_name}
                ON masterdatabase.contract_tree_information CASCADE
            """))
            print(f"   ✅ Eliminado: {trigger_name}")

            # Eliminar también la función si existe
            func_name = trigger_name.replace('trigger_', '')
            conn.execute(text(f"""
                DROP FUNCTION IF EXISTS masterdatabase.{func_name}() CASCADE
            """))
else:
    print("\n✅ No hay triggers en contract_tree_information")

print("\n✅ Limpieza de triggers completada")


🧹 LIMPIANDO TRIGGERS OBSOLETOS

📋 Triggers actuales en masterdatabase:
            tabla                      trigger_name evento
inventory_metrics trigger_sync_inventory_to_current INSERT
inventory_metrics trigger_sync_inventory_to_current UPDATE

✅ No hay triggers en contract_tree_information

✅ Limpieza de triggers completada


In [47]:
from core.db import get_engine
from core.libs import pd
from sqlalchemy import text

engine = get_engine()

print("="*70)
print("🔍 AUDITORÍA COMPLETA DE ARQUITECTURA - MASTERDATABASE")
print("="*70)

# ============================================================
# 1️⃣ TABLAS Y VISTAS
# ============================================================
print("\n" + "="*70)
print("1️⃣ TABLAS Y VISTAS")
print("="*70)

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            table_name,
            table_type
        FROM information_schema.tables
        WHERE table_schema = 'masterdatabase'
          AND table_name IN (
              'contract_tree_information',
              'inventory_metrics',
              'inventory_metrics_current',
              'survival_current',
              'survival_timeseries',
              'farmer_personal_information',
              'contract_allocation'
          )
        ORDER BY
            CASE table_type
                WHEN 'BASE TABLE' THEN 1
                WHEN 'VIEW' THEN 2
                ELSE 3
            END,
            table_name
    """))
    tables_views = pd.DataFrame(result.fetchall(), columns=result.keys())

print("\n📊 Objetos principales:")
print(tables_views.to_string(index=False))

# Contar registros
print("\n📈 Conteo de registros:")
for _, row in tables_views.iterrows():
    if row['table_type'] in ['BASE TABLE', 'VIEW']:
        try:
            with engine.connect() as conn:
                result = conn.execute(text(f"SELECT COUNT(*) as total FROM masterdatabase.{row['table_name']}"))
                count = result.fetchone()[0]
            print(f"   {row['table_name']:<35} {count:>6} registros")
        except Exception as e:
            print(f"   {row['table_name']:<35} ERROR")

# ============================================================
# 2️⃣ TRIGGERS
# ============================================================
print("\n" + "="*70)
print("2️⃣ TRIGGERS ACTIVOS")
print("="*70)

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            t.event_object_table as tabla_origen,
            t.trigger_name,
            t.event_manipulation as evento,
            t.action_timing as momento
        FROM information_schema.triggers t
        WHERE t.trigger_schema = 'masterdatabase'
        ORDER BY t.event_object_table, t.trigger_name
    """))
    triggers = pd.DataFrame(result.fetchall(), columns=result.keys())

if len(triggers) > 0:
    print("\n⚙️ Triggers encontrados:")
    print(triggers.to_string(index=False))

    print("\n📋 Análisis por tabla:")
    for tabla in triggers['tabla_origen'].unique():
        tabla_triggers = triggers[triggers['tabla_origen'] == tabla]
        print(f"\n   📁 {tabla}:")
        for _, trig in tabla_triggers.iterrows():
            print(f"      → {trig['evento']} ejecuta {trig['trigger_name']}")
else:
    print("\n⚠️ No se encontraron triggers activos")

# ============================================================
# 3️⃣ FUNCIONES DE TRIGGERS
# ============================================================
print("\n" + "="*70)
print("3️⃣ FUNCIONES DE SINCRONIZACIÓN")
print("="*70)

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            p.proname as nombre_funcion,
            pg_get_function_identity_arguments(p.oid) as argumentos
        FROM pg_proc p
        JOIN pg_namespace n ON p.pronamespace = n.oid
        WHERE n.nspname = 'masterdatabase'
          AND (p.proname LIKE '%sync%' OR p.proname LIKE '%trigger%')
        ORDER BY p.proname
    """))
    functions = pd.DataFrame(result.fetchall(), columns=result.keys())

if len(functions) > 0:
    print(f"\n🔧 Funciones encontradas: {len(functions)}")
    for _, func in functions.iterrows():
        print(f"   - {func['nombre_funcion']}({func['argumentos']})")
else:
    print("\nℹ️ No se encontraron funciones de sincronización")

# ============================================================
# 4️⃣ CONSTRAINTS
# ============================================================
print("\n" + "="*70)
print("4️⃣ CONSTRAINTS PRINCIPALES")
print("="*70)

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            tc.table_name,
            tc.constraint_name,
            tc.constraint_type,
            string_agg(kcu.column_name, ', ') as columnas
        FROM information_schema.table_constraints tc
        LEFT JOIN information_schema.key_column_usage kcu
            ON tc.constraint_name = kcu.constraint_name
            AND tc.table_schema = kcu.table_schema
        WHERE tc.table_schema = 'masterdatabase'
          AND tc.table_name IN (
              'contract_tree_information',
              'inventory_metrics',
              'inventory_metrics_current'
          )
        GROUP BY tc.table_name, tc.constraint_name, tc.constraint_type
        ORDER BY tc.table_name, tc.constraint_type
    """))
    constraints = pd.DataFrame(result.fetchall(), columns=result.keys())

if len(constraints) > 0:
    print("\n🔗 Constraints por tabla:")
    for tabla in constraints['table_name'].unique():
        tabla_constraints = constraints[constraints['table_name'] == tabla]
        print(f"\n   📁 {tabla}:")
        for _, cons in tabla_constraints.iterrows():
            print(f"      {cons['constraint_type']}: {cons['constraint_name']} en ({cons['columnas']})")
else:
    print("\nℹ️ No se encontraron constraints")

# ============================================================
# 5️⃣ ÍNDICES
# ============================================================
print("\n" + "="*70)
print("5️⃣ ÍNDICES PRINCIPALES")
print("="*70)

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            tablename,
            indexname,
            indexdef
        FROM pg_indexes
        WHERE schemaname = 'masterdatabase'
          AND tablename IN (
              'contract_tree_information',
              'inventory_metrics',
              'inventory_metrics_current'
          )
        ORDER BY tablename, indexname
    """))
    indexes = pd.DataFrame(result.fetchall(), columns=result.keys())

if len(indexes) > 0:
    print("\n📑 Índices por tabla:")
    for tabla in indexes['tablename'].unique():
        tabla_indexes = indexes[indexes['tablename'] == tabla]
        print(f"\n   📁 {tabla}: ({len(tabla_indexes)} índices)")
        for _, idx in tabla_indexes.iterrows():
            print(f"      - {idx['indexname']}")
else:
    print("\nℹ️ No se encontraron índices")

# ============================================================
# 6️⃣ VISTAS Y SUS DEPENDENCIAS
# ============================================================
print("\n" + "="*70)
print("6️⃣ VISTAS")
print("="*70)

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT
            table_name as vista,
            view_definition
        FROM information_schema.views
        WHERE table_schema = 'masterdatabase'
          AND table_name = 'survival_current'
    """))
    views = pd.DataFrame(result.fetchall(), columns=result.keys())

if len(views) > 0:
    print("\n👁️ Vistas activas:")
    for _, view in views.iterrows():
        print(f"\n   📊 {view['vista']}:")
        definition = view['view_definition'].lower()

        # Extraer tablas referenciadas
        tables_ref = []
        if 'inventory_metrics_current' in definition:
            tables_ref.append('inventory_metrics_current')
        if 'inventory_metrics' in definition and 'inventory_metrics_current' not in definition:
            tables_ref.append('inventory_metrics')

        if tables_ref:
            print("      Depende de:")
            for table in tables_ref:
                print(f"         - {table}")
else:
    print("\nℹ️ No se encontraron vistas")

# ============================================================
# 7️⃣ VERIFICACIÓN DE ARQUITECTURA
# ============================================================
print("\n" + "="*70)
print("7️⃣ VERIFICACIÓN DE ARQUITECTURA")
print("="*70)

print("""
📐 ARQUITECTURA ESPERADA:

   CTI (manual) → IM (histórica) →[trigger]→ IMC (current) →[VIEW]→ survival_current
""")

print("\n🔍 Estado actual:")

# 1. Trigger IM → IMC
im_trigger = triggers[triggers['tabla_origen'] == 'inventory_metrics']
if len(im_trigger) > 0:
    print("   ✅ Trigger IM → IMC presente")
    for _, t in im_trigger.iterrows():
        print(f"      └─ {t['trigger_name']} ({t['evento']})")
else:
    print("   ❌ Trigger IM → IMC FALTANTE")

# 2. survival_current es VIEW
sc_is_view = tables_views[
    (tables_views['table_name'] == 'survival_current') &
    (tables_views['table_type'] == 'VIEW')
]
if len(sc_is_view) > 0:
    print("   ✅ survival_current es VIEW")
else:
    print("   ❌ survival_current NO es VIEW")

# 3. No hay triggers en CTI
cti_triggers = triggers[triggers['tabla_origen'] == 'contract_tree_information']
if len(cti_triggers) == 0:
    print("   ✅ No hay triggers en CTI (correcto)")
else:
    print(f"   ⚠️ Hay {len(cti_triggers)} trigger(s) en CTI:")
    for _, t in cti_triggers.iterrows():
        print(f"      └─ {t['trigger_name']} (verificar si es obsoleto)")

# 4. IMC tiene UNIQUE
imc_unique = constraints[
    (constraints['table_name'] == 'inventory_metrics_current') &
    (constraints['constraint_type'] == 'UNIQUE')
]
if len(imc_unique) > 0:
    print("   ✅ IMC tiene UNIQUE constraint")
else:
    print("   ⚠️ IMC sin UNIQUE constraint")

# 5. Conteos de registros
with engine.connect() as conn:
    im_count = conn.execute(text("SELECT COUNT(*) FROM masterdatabase.inventory_metrics")).fetchone()[0]
    imc_count = conn.execute(text("SELECT COUNT(*) FROM masterdatabase.inventory_metrics_current")).fetchone()[0]
    sc_count = conn.execute(text("SELECT COUNT(*) FROM masterdatabase.survival_current")).fetchone()[0]

print(f"\n📊 Conteos:")
print(f"   - inventory_metrics: {im_count}")
print(f"   - inventory_metrics_current: {imc_count}")
print(f"   - survival_current (VIEW): {sc_count}")

if im_count == imc_count:
    print("   ✅ IM e IMC tienen misma cantidad (sincronizadas)")
else:
    print(f"   ℹ️ IM tiene {im_count - imc_count} registros más que IMC (normal si hay históricos)")

print("\n" + "="*70)
print("✅ AUDITORÍA COMPLETADA")
print("="*70)

💻 Conectado a la base de datos helloworldtree
🔍 AUDITORÍA COMPLETA DE ARQUITECTURA - MASTERDATABASE

1️⃣ TABLAS Y VISTAS

📊 Objetos principales:
                 table_name table_type
        contract_allocation BASE TABLE
  contract_tree_information BASE TABLE
farmer_personal_information BASE TABLE
          inventory_metrics BASE TABLE
  inventory_metrics_current BASE TABLE
           survival_current       VIEW

📈 Conteo de registros:
   contract_allocation                    462 registros
   contract_tree_information              462 registros
   farmer_personal_information            385 registros
   inventory_metrics                      424 registros
   inventory_metrics_current              424 registros
   survival_current                       198 registros

2️⃣ TRIGGERS ACTIVOS

⚙️ Triggers encontrados:
     tabla_origen                      trigger_name evento momento
inventory_metrics trigger_sync_inventory_to_current INSERT   AFTER
inventory_metrics trigger_sync_inventory

In [46]:
from core.db import get_engine
from sqlalchemy import text

engine = get_engine()

print("🧹 Eliminando función obsoleta...")

with engine.begin() as conn:
    conn.execute(text("""
        DROP FUNCTION IF EXISTS masterdatabase.sync_contract_to_imc() CASCADE
    """))
    print("✅ sync_contract_to_imc() eliminada")

# Verificar
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT p.proname
        FROM pg_proc p
        JOIN pg_namespace n ON p.pronamespace = n.oid
        WHERE n.nspname = 'masterdatabase'
          AND p.proname LIKE '%sync%'
    """))
    functions = result.fetchall()

print("\n📋 Funciones restantes:")
for func in functions:
    print(f"   - {func[0]}()")

print("\n✅ LIMPIEZA COMPLETADA")

💻 Conectado a la base de datos helloworldtree
🧹 Eliminando función obsoleta...
✅ sync_contract_to_imc() eliminada

📋 Funciones restantes:
   - sync_inventory_to_current()

✅ LIMPIEZA COMPLETADA


# ==============================================================
# 🔗 SECCIÓN: RELACIONES FPI (farmer_personal_information)
# ==============================================================

print("======================================================================")
print("🔍 RELACIONES FPI → CONTRACT HEADER → MISSING FIELDS")
print("======================================================================")


In [49]:
# ==============================================================
# 1️⃣ VISTA AUXILIAR - EXPANSIÓN DE CONTRACT_CODES
# ==============================================================

with engine.begin() as conn:
    conn.exec_driver_sql("""
        CREATE OR REPLACE VIEW masterdatabase.vw_fpi_contracts AS
        SELECT
            fpi.farmer_number,
            unnest(fpi.contract_codes) AS contract_code
        FROM masterdatabase.farmer_personal_information fpi;
    """)
print("✅ Vista creada/actualizada: masterdatabase.vw_fpi_contracts")

# ==============================================================
# 2️⃣ AUDITORÍA DE RELACIONES
# ==============================================================

print("\n📋 Verificación de correspondencias...")

query_relations = """
SELECT
    COUNT(DISTINCT fpi.farmer_number)          AS total_farmers,
    COUNT(DISTINCT ch.contract_code)           AS total_contracts,
    COUNT(DISTINCT v.contract_code)            AS linked_contracts,
    (COUNT(DISTINCT ch.contract_code) - COUNT(DISTINCT v.contract_code)) AS unlinked_contracts
FROM masterdatabase.contract_header ch
LEFT JOIN masterdatabase.vw_fpi_contracts v USING (contract_code)
LEFT JOIN masterdatabase.farmer_personal_information fpi
       ON fpi.farmer_number = ch.farmer_number;
"""
df_relations = pd.read_sql(query_relations, engine)
display(df_relations)

# ==============================================================
# 3️⃣ LISTADO DE CONTRATOS SIN VÍNCULO EN FPI
# ==============================================================

print("\n🚨 Contratos sin correspondencia en FPI:")

query_missing = """
SELECT ch.contract_code, ch.farmer_number, ch.contract_name
FROM masterdatabase.contract_header ch
LEFT JOIN masterdatabase.vw_fpi_contracts v USING (contract_code)
WHERE v.contract_code IS NULL
ORDER BY ch.contract_code;
"""
df_missing = pd.read_sql(query_missing, engine)
display(df_missing)

# ==============================================================
# 4️⃣ INTEGRACIÓN CON CFI_MISSING_FIELDS (VALIDACIÓN COMPLEMENTARIA)
# ==============================================================

print("\n📎 Campos faltantes por contrato y su farmer asociado:")

query_cfi = """
SELECT
    cmf.contract_code,
    cmf.field,
    fpi.representative,
    fpi.email,
    fpi.farmer_number
FROM masterdatabase.cfi_missing_fields cmf
LEFT JOIN masterdatabase.vw_fpi_contracts v USING (contract_code)
LEFT JOIN masterdatabase.farmer_personal_information fpi USING (farmer_number)
ORDER BY cmf.contract_code, cmf.field;
"""
df_cfi = pd.read_sql(query_cfi, engine)
display(df_cfi)

print("\n======================================================================")
print("✅ RELACIONES FPI - AUDITORÍA COMPLETADA")
print("======================================================================")


✅ Vista creada/actualizada: masterdatabase.vw_fpi_contracts

📋 Verificación de correspondencias...


,total_farmers,total_contracts,linked_contracts,unlinked_contracts
0,357,421,421,0



🚨 Contratos sin correspondencia en FPI:


,contract_code,farmer_number,contract_name



📎 Campos faltantes por contrato y su farmer asociado:


,contract_code,field,representative,email,farmer_number
0,MX0053,address,None,NA,40048
1,MX0053,email,None,NA,40048
2,MX0053,phone,None,NA,40048
3,MX0053,representative,None,NA,40048
4,MX0054,address,None,NA,40049
5,MX0054,email,None,NA,40049
6,MX0054,representative,None,NA,40049
7,MX0055,address,Casto De la Cruz Francisco,delacruz5castro@gmail.com,40050
8,MX0055,phone,Casto De la Cruz Francisco,delacruz5castro@gmail.com,40050
9,MX0055,shipping_address,Casto De la Cruz Francisco,delacruz5castro@gmail.com,40050



✅ RELACIONES FPI - AUDITORÍA COMPLETADA


In [50]:
# ==============================================================
# 🔗 SECCIÓN: RELACIÓN CTI ↔ FPI
# ==============================================================

print("======================================================================")
print("🔍 RELACIÓN ENTRE contract_tree_information Y farmer_personal_information")
print("======================================================================")

# ==============================================================
# 1️⃣ CREAR VISTA AUXILIAR - EXPANSIÓN FPI PARA VINCULAR CTI
# ==============================================================

with engine.begin() as conn:
    conn.exec_driver_sql("""
        CREATE OR REPLACE VIEW masterdatabase.vw_cti_fpi_link AS
        SELECT
            cti.contract_code,
            cti.etp_year,
            cti.status,
            fpi.farmer_number,
            fpi.representative,
            fpi.email,
            fpi.phone,
            fpi.address,
            fpi.shipping_address
        FROM masterdatabase.contract_tree_information cti
        LEFT JOIN masterdatabase.farmer_personal_information fpi
            ON cti.contract_code = ANY(fpi.contract_codes);
    """)
print("✅ Vista creada/actualizada: masterdatabase.vw_cti_fpi_link")

# ==============================================================
# 2️⃣ AUDITORÍA DE ENLACES CTI–FPI
# ==============================================================

print("\n📋 Verificación de correspondencias CTI–FPI:")

query_audit = """
SELECT
    COUNT(DISTINCT cti.contract_code) AS total_cti,
    COUNT(DISTINCT link.contract_code) AS linked_cti,
    COUNT(DISTINCT cti.contract_code) - COUNT(DISTINCT link.contract_code) AS unlinked_cti,
    COUNT(DISTINCT link.farmer_number) AS distinct_farmers
FROM masterdatabase.contract_tree_information cti
LEFT JOIN masterdatabase.vw_cti_fpi_link link USING (contract_code);
"""
df_cti_audit = pd.read_sql(query_audit, engine)
display(df_cti_audit)

# ==============================================================
# 3️⃣ LISTADO DE CONTRATOS SIN FARMER VINCULADO
# ==============================================================

print("\n🚨 Contratos en CTI sin correspondencia en FPI:")

query_unlinked = """
SELECT
    cti.contract_code,
    cti.etp_year,
    cti.status
FROM masterdatabase.contract_tree_information cti
LEFT JOIN masterdatabase.vw_cti_fpi_link link USING (contract_code)
WHERE link.farmer_number IS NULL
ORDER BY cti.contract_code;
"""
df_unlinked = pd.read_sql(query_unlinked, engine)
display(df_unlinked)

# ==============================================================
# 4️⃣ VALIDACIÓN CRUZADA (duplicados o conflictos)
# ==============================================================

print("\n🔎 Contratos con más de un farmer asociado (posibles conflictos):")

query_conflicts = """
SELECT
    contract_code,
    COUNT(DISTINCT farmer_number) AS farmer_count,
    ARRAY_AGG(DISTINCT farmer_number) AS farmers
FROM masterdatabase.vw_cti_fpi_link
GROUP BY contract_code
HAVING COUNT(DISTINCT farmer_number) > 1
ORDER BY contract_code;
"""
df_conflicts = pd.read_sql(query_conflicts, engine)
display(df_conflicts)

print("\n======================================================================")
print("✅ RELACIÓN CTI ↔ FPI - AUDITORÍA COMPLETADA")
print("======================================================================")


🔍 RELACIÓN ENTRE contract_tree_information Y farmer_personal_information
✅ Vista creada/actualizada: masterdatabase.vw_cti_fpi_link

📋 Verificación de correspondencias CTI–FPI:


,total_cti,linked_cti,unlinked_cti,distinct_farmers
0,462,462,0,385



🚨 Contratos en CTI sin correspondencia en FPI:


,contract_code,etp_year,status
0,CR0014,2016,Out of Program



🔎 Contratos con más de un farmer asociado (posibles conflictos):


,contract_code,farmer_count,farmers



✅ RELACIÓN CTI ↔ FPI - AUDITORÍA COMPLETADA


In [52]:
# ==============================================================
# 🪦 SECCIÓN 7️⃣ — DEPRECACIÓN CONTROLADA DE survival_timeseries
# ==============================================================

print("======================================================================")
print("🔍 PREPARACIÓN PARA DEPRECACIÓN DE survival_timeseries")
print("======================================================================")

# Paso 1️⃣ — Auditoría previa y respaldo
query_audit = """
SELECT COUNT(*) AS registros,
       MIN(inventory_date) AS min_date,
       MAX(inventory_date) AS max_date
FROM masterdatabase.survival_timeseries;
"""
df = pd.read_sql(query_audit, engine)
display(df)

with engine.begin() as conn:
    conn.exec_driver_sql("""
        CREATE TABLE IF NOT EXISTS masterdatabase.survival_timeseries_backup AS
        SELECT *, now() AS backup_timestamp
        FROM masterdatabase.survival_timeseries;
    """)
print("✅ Respaldo creado: masterdatabase.survival_timeseries_backup")

🔍 PREPARACIÓN PARA DEPRECACIÓN DE survival_timeseries


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "masterdatabase.survival_timeseries" does not exist
LINE 5: FROM masterdatabase.survival_timeseries;
             ^

[SQL: 
SELECT COUNT(*) AS registros,
       MIN(inventory_date) AS min_date,
       MAX(inventory_date) AS max_date
FROM masterdatabase.survival_timeseries;
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [53]:

# Paso 2️⃣ — Desvincular triggers asociados (si existieran)

print("\n🧹 Eliminando triggers asociados a survival_timeseries...")

with engine.begin() as conn:
    conn.exec_driver_sql("""
        DROP TRIGGER IF EXISTS trigger_log_survival_timeseries
        ON masterdatabase.inventory_metrics_current;
    """)
print("✅ Trigger trigger_log_survival_timeseries eliminado (si existía)")

# Paso 3️⃣ — Eliminación controlada de la tabla

print("\n🔥 Eliminando tabla survival_timeseries...")

with engine.begin() as conn:
    conn.exec_driver_sql("""
        DROP TABLE IF EXISTS masterdatabase.survival_timeseries;
    """)
print("✅ Tabla eliminada: masterdatabase.survival_timeseries (respaldo conservado)")


🧹 Eliminando triggers asociados a survival_timeseries...
✅ Trigger trigger_log_survival_timeseries eliminado (si existía)

🔥 Eliminando tabla survival_timeseries...
✅ Tabla eliminada: masterdatabase.survival_timeseries (respaldo conservado)


In [56]:
# Paso 4️⃣ — Verificación post-deprecación (versión segura SQLAlchemy 2.x)

from sqlalchemy import text

print("\n🔎 Verificando integridad tras deprecación:")

query_verify = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'masterdatabase'
  AND table_name LIKE 'survival%';
"""

with engine.connect() as conn:
    df_verify = pd.read_sql(text(query_verify), conn)

display(df_verify)

print("\n✅ Solo debería permanecer: survival_current (VIEW)")
print("======================================================================")
print("🏁 DEPRECACIÓN DE survival_timeseries COMPLETADA")
print("======================================================================")



🔎 Verificando integridad tras deprecación:


,table_name
0,survival_current
1,survival_current_backup
2,survival_timeseries_backup



✅ Solo debería permanecer: survival_current (VIEW)
🏁 DEPRECACIÓN DE survival_timeseries COMPLETADA


In [57]:
# ==============================================================
# 🔄 SECCIÓN 8️⃣ — REDEFINICIÓN DE survival_current POST-DEPRECACIÓN
# ==============================================================

with engine.begin() as conn:
    conn.exec_driver_sql("""
        CREATE OR REPLACE VIEW masterdatabase.survival_current AS
        SELECT
            imc.contract_code,
            imc.survival AS current_survival_pct,
            imc.total_trees AS current_surviving_trees,
            CASE
                WHEN imc.type_of_metric = 'placeholder' THEN 'Placeholder (IMC)'
                WHEN imc.inventory_year IS NOT NULL THEN 'Inventory ' || imc.inventory_year::text
                ELSE 'IMC (unspecified)'
            END AS survival_metric_source,
            imc.inventory_year,
            imc.inventory_date,
            imc.mortality
        FROM masterdatabase.inventory_metrics_current imc
        WHERE imc.survival IS NOT NULL;
    """)
print("✅ Vista survival_current actualizada: ahora lee exclusivamente de inventory_metrics_current")


✅ Vista survival_current actualizada: ahora lee exclusivamente de inventory_metrics_current


In [58]:
print("======================================================================")
print("🔍 VERIFICACIÓN POST-REDEFINICIÓN DE survival_current")
print("======================================================================")

query_check = """
SELECT
    COUNT(*) AS total_records,
    COUNT(DISTINCT contract_code) AS unique_contracts,
    MIN(inventory_year) AS min_year,
    MAX(inventory_year) AS max_year
FROM masterdatabase.survival_current;
"""
df_check = pd.read_sql(query_check, engine)
display(df_check)

print("\n✅ survival_current ahora depende directamente de inventory_metrics_current")
print("   y refleja los registros más recientes por contrato.")
print("======================================================================")


🔍 VERIFICACIÓN POST-REDEFINICIÓN DE survival_current


,total_records,unique_contracts,min_year,max_year
0,198,198,2021,2025



✅ survival_current ahora depende directamente de inventory_metrics_current
   y refleja los registros más recientes por contrato.


In [59]:
print("🔗 Confirmando dependencias de survival_current...")

query_dependencies = """
SELECT
    dependent_view.relname AS view_name,
    source_table.relname AS source_table
FROM pg_depend d
JOIN pg_rewrite r ON d.objid = r.oid
JOIN pg_class dependent_view ON r.ev_class = dependent_view.oid
JOIN pg_class source_table ON d.refobjid = source_table.oid
JOIN pg_namespace n ON n.oid = dependent_view.relnamespace
WHERE n.nspname = 'masterdatabase'
  AND dependent_view.relname = 'survival_current';
"""
df_dep = pd.read_sql(query_dependencies, engine)
display(df_dep)

print("\n✅ Dependencias verificadas correctamente.")


🔗 Confirmando dependencias de survival_current...


,view_name,source_table
0,survival_current,survival_current
1,survival_current,inventory_metrics_current
2,survival_current,inventory_metrics_current
3,survival_current,inventory_metrics_current
4,survival_current,inventory_metrics_current
5,survival_current,inventory_metrics_current
6,survival_current,inventory_metrics_current
7,survival_current,inventory_metrics_current



✅ Dependencias verificadas correctamente.


In [62]:
# ==============================================================
# 🧾 SECCIÓN 10️⃣ — EXPORT DE TODAS LAS RELACIONES (FÍSICAS + LÓGICAS)
# ==============================================================

from sqlalchemy import text
import pandas as pd
from core.paths import DATABASE_EXPORTS_DIR, safe_mkdir

print("======================================================================")
print("🔍 RECOLECTANDO RELACIONES DEL ESQUEMA masterdatabase")
print("======================================================================")

# Asegurarse de que la carpeta existe
safe_mkdir(DATABASE_EXPORTS_DIR)

# 1️⃣ Relaciones físicas (FOREIGN KEYs)
query_physical = text("""
SELECT
    conname AS constraint_name,
    conrelid::regclass AS child_table,
    confrelid::regclass AS parent_table,
    'physical' AS relationship_type,
    'FOREIGN KEY' AS description
FROM pg_constraint
WHERE contype = 'f'
  AND connamespace = (SELECT oid FROM pg_namespace WHERE nspname = 'masterdatabase');
""")

# 2️⃣ Relaciones lógicas (vistas, dependencias)
query_logical = text("""
SELECT
    dependent_view.relname AS child_table,
    source_table.relname AS parent_table,
    'logical' AS relationship_type,
    CASE
        WHEN dependent_view.relkind = 'v' THEN 'VIEW: ' || dependent_view.relname
        WHEN dependent_view.relkind = 'r' THEN 'TABLE: ' || dependent_view.relname
        ELSE 'OTHER'
    END AS description
FROM pg_depend d
JOIN pg_rewrite r ON d.objid = r.oid
JOIN pg_class dependent_view ON r.ev_class = dependent_view.oid
JOIN pg_class source_table ON d.refobjid = source_table.oid
JOIN pg_namespace n ON n.oid = dependent_view.relnamespace
WHERE n.nspname = 'masterdatabase'
ORDER BY dependent_view.relname;
""")

# 3️⃣ Triggers activos
query_triggers = text("""
SELECT
    event_object_table AS parent_table,
    trigger_name AS child_table,
    'trigger' AS relationship_type,
    action_statement AS description
FROM information_schema.triggers
WHERE trigger_schema = 'masterdatabase';
""")

# Ejecutar consultas
with engine.connect() as conn:
    df_phys = pd.read_sql(query_physical, conn)
    df_log = pd.read_sql(query_logical, conn)
    df_trig = pd.read_sql(query_triggers, conn)

# Normalizar y combinar
df_phys = df_phys.rename(columns={"constraint_name": "link_name"})
df_log["link_name"] = None
df_trig["link_name"] = None

df_all = pd.concat([df_phys, df_log, df_trig], ignore_index=True)
display(df_all)

# 4️⃣ Exportar a DatabaseExports
export_xlsx = DATABASE_EXPORTS_DIR / "masterdatabase_dependencies_full.xlsx"
export_csv = DATABASE_EXPORTS_DIR / "masterdatabase_dependencies_full.csv"

df_all.to_excel(export_xlsx, index=False)
df_all.to_csv(export_csv, index=False)
print(f"✅ Export completado en:\n📘 Excel: {export_xlsx}\n📄 CSV:   {export_csv}")
print("======================================================================")
# ==============================================================
# 🧾 SECCIÓN 10️⃣ — EXPORT DE TODAS LAS RELACIONES (FÍSICAS + LÓGICAS)
# ==============================================================

from sqlalchemy import text
import pandas as pd
from core.paths import DATABASE_EXPORTS_DIR, safe_mkdir

print("======================================================================")
print("🔍 RECOLECTANDO RELACIONES DEL ESQUEMA masterdatabase")
print("======================================================================")

# Asegurarse de que la carpeta existe
safe_mkdir(DATABASE_EXPORTS_DIR)

# 1️⃣ Relaciones físicas (FOREIGN KEYs)
query_physical = text("""
SELECT
    conname AS constraint_name,
    conrelid::regclass AS child_table,
    confrelid::regclass AS parent_table,
    'physical' AS relationship_type,
    'FOREIGN KEY' AS description
FROM pg_constraint
WHERE contype = 'f'
  AND connamespace = (SELECT oid FROM pg_namespace WHERE nspname = 'masterdatabase');
""")

# 2️⃣ Relaciones lógicas (vistas, dependencias)
query_logical = text("""
SELECT
    dependent_view.relname AS child_table,
    source_table.relname AS parent_table,
    'logical' AS relationship_type,
    CASE
        WHEN dependent_view.relkind = 'v' THEN 'VIEW: ' || dependent_view.relname
        WHEN dependent_view.relkind = 'r' THEN 'TABLE: ' || dependent_view.relname
        ELSE 'OTHER'
    END AS description
FROM pg_depend d
JOIN pg_rewrite r ON d.objid = r.oid
JOIN pg_class dependent_view ON r.ev_class = dependent_view.oid
JOIN pg_class source_table ON d.refobjid = source_table.oid
JOIN pg_namespace n ON n.oid = dependent_view.relnamespace
WHERE n.nspname = 'masterdatabase'
ORDER BY dependent_view.relname;
""")

# 3️⃣ Triggers activos
query_triggers = text("""
SELECT
    event_object_table AS parent_table,
    trigger_name AS child_table,
    'trigger' AS relationship_type,
    action_statement AS description
FROM information_schema.triggers
WHERE trigger_schema = 'masterdatabase';
""")

# Ejecutar consultas
with engine.connect() as conn:
    df_phys = pd.read_sql(query_physical, conn)
    df_log = pd.read_sql(query_logical, conn)
    df_trig = pd.read_sql(query_triggers, conn)

# Normalizar y combinar
df_phys = df_phys.rename(columns={"constraint_name": "link_name"})
df_log["link_name"] = None
df_trig["link_name"] = None

df_all = pd.concat([df_phys, df_log, df_trig], ignore_index=True)
display(df_all)

# 4️⃣ Exportar a DatabaseExports
export_xlsx = DATABASE_EXPORTS_DIR / "masterdatabase_dependencies_full.xlsx"
export_csv = DATABASE_EXPORTS_DIR / "masterdatabase_dependencies_full.csv"

df_all.to_excel(export_xlsx, index=False)
df_all.to_csv(export_csv, index=False)
print(f"✅ Export completado en:\n📘 Excel: {export_xlsx}\n📄 CSV:   {export_csv}")
print("======================================================================")


🔍 RECOLECTANDO RELACIONES DEL ESQUEMA masterdatabase


,link_name,child_table,parent_table,relationship_type,description
0,ca_cti_fk,masterdatabase.contract_allocation,masterdatabase.contract_tree_information,physical,FOREIGN KEY
1,contract_header_farmer_number_fkey,masterdatabase.contract_header,masterdatabase.farmer_personal_information,physical,FOREIGN KEY
2,None,fpi_contracts_expanded,fpi_contracts_expanded,logical,VIEW: fpi_contracts_expanded
3,None,fpi_contracts_expanded,farmer_personal_information,logical,VIEW: fpi_contracts_expanded
4,None,mv_atlas_years_fixed,mv_states_by_etp_year_qgis,logical,OTHER
5,None,mv_atlas_years_fixed,atlas_bbox,logical,OTHER
6,None,mv_atlas_years_fixed,mv_atlas_years_fixed,logical,OTHER
7,None,mv_states_by_etp_year_qgis,vw_contracts_map_metrics,logical,OTHER
8,None,mv_states_by_etp_year_qgis,vw_contracts_map_metrics,logical,OTHER
9,None,mv_states_by_etp_year_qgis,WorldTree_States,logical,OTHER


✅ Export completado en:
📘 Excel: C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\masterdatabase_dependencies_full.xlsx
📄 CSV:   C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\masterdatabase_dependencies_full.csv
🔍 RECOLECTANDO RELACIONES DEL ESQUEMA masterdatabase


,link_name,child_table,parent_table,relationship_type,description
0,ca_cti_fk,masterdatabase.contract_allocation,masterdatabase.contract_tree_information,physical,FOREIGN KEY
1,contract_header_farmer_number_fkey,masterdatabase.contract_header,masterdatabase.farmer_personal_information,physical,FOREIGN KEY
2,None,fpi_contracts_expanded,fpi_contracts_expanded,logical,VIEW: fpi_contracts_expanded
3,None,fpi_contracts_expanded,farmer_personal_information,logical,VIEW: fpi_contracts_expanded
4,None,mv_atlas_years_fixed,mv_states_by_etp_year_qgis,logical,OTHER
5,None,mv_atlas_years_fixed,atlas_bbox,logical,OTHER
6,None,mv_atlas_years_fixed,mv_atlas_years_fixed,logical,OTHER
7,None,mv_states_by_etp_year_qgis,vw_contracts_map_metrics,logical,OTHER
8,None,mv_states_by_etp_year_qgis,vw_contracts_map_metrics,logical,OTHER
9,None,mv_states_by_etp_year_qgis,WorldTree_States,logical,OTHER


✅ Export completado en:
📘 Excel: C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\masterdatabase_dependencies_full.xlsx
📄 CSV:   C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\masterdatabase_dependencies_full.csv


In [15]:
from core.db import get_engine
from core.libs import pd

engine = get_engine()

def get_headers(table_name, schema="masterdatabase"):
    query = f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = '{schema}'
          AND table_name = '{table_name}'
        ORDER BY ordinal_position;
    """
    return pd.read_sql(query, engine)

tables = [
    ("CTI", "contract_tree_information"),
    ("FPI", "farmer_personal_information"),
    ("CA",  "contract_allocation"),
    ("IMC", "inventory_metrics_current"),
    ("IM",  "inventory_metrics"),
]

for alias, table in tables:
    df_cols = get_headers(table)
    print(f"\n--- {alias} ({table}) ---")
    for col in df_cols["column_name"].tolist():
        print(col)


💻 Conectado a la base de datos helloworldtree

--- CTI (contract_tree_information) ---
contract_amendment
contract_code
etp_year
harvest_year
land_location
latitude
longitude
planted
planting_obligation
planting_year
region
species
strain
trees_contract
loaded_at
planting_date
status
Filter
land_location_pro
etp_year_backup

--- FPI (farmer_personal_information) ---
farmer_number
representative
phone
email
address
shipping_address
contract_codes
contract_name

--- CA (contract_allocation) ---
contract_code
usa_allocation_pct
usa_trees_contracted
usa_trees_planted
canada_allocation_pct
canada_2017_trees
canada_2017_allocation_pct
canada_trees_contracted
total_can_allocation
loaded_at
etp_type
contracted_cop
planted_cop
contracted_etp
planted_etp
surviving_etp
surviving_cop

--- IMC (inventory_metrics_current) ---
rel_path
contract_code
planting_year
tree_age
inventory_year
inventory_date
tht_mean
tht_std
mht_mean
mht_std
dbh_mean
dbh_std
noncom_dbh_count
doyle_bf_mean
doyle_bf_std
doyle

In [14]:
def get_headers_with_types(table_name, schema="masterdatabase"):
    query = f"""
        SELECT
            column_name,
            data_type
        FROM information_schema.columns
        WHERE table_schema = '{schema}'
          AND table_name = '{table_name}'
        ORDER BY ordinal_position;
    """
    return pd.read_sql(query, engine)

# Ejemplo para IMC
df_im = get_headers_with_types("inventory_metrics")
df_im.head(27)


,column_name,data_type
0,rel_path,text
1,contract_code,text
2,tree_age,text
3,inventory_year,bigint
4,inventory_date,date
5,tht_mean,double precision
6,tht_std,double precision
7,mht_mean,double precision
8,mht_std,double precision
9,dbh_mean,double precision


In [ ]:
cols = [
    "survival",
    "mht_pct_of_target",
    "dbh_pct_of_target",
    "projected_dbh",
    "projected_doyle_bf",
    "mortality",
]

df = pd.read_sql(
    f"""
    SELECT {", ".join(cols)}
    FROM masterdatabase.inventory_metrics_current
    LIMIT 5;
    """,
    engine
)

In [9]:

df = df.dropna(how="all", subset=cols)  # dropea filas donde TODO es None


print(df.head())


Empty DataFrame
Columns: [survival, mht_pct_of_target, dbh_pct_of_target, projected_dbh, projected_doyle_bf, mortality]
Index: []


Creación de VIEWS maestras para análisis operacional

In [16]:
"""
master_views_generator.py
Crea 3 VIEWS en PostgreSQL para análisis operacional:
1. master_join_operational: Snapshot actual (FPI+CTI+CA+IMC+SURV)
2. join_master: Relaciones contract-farmer
3. master_join_expanded: Desglose completo con inventory_metrics
"""

from core.db import get_engine
from core.libs import pd, text
from core.paths import DATABASE_EXPORTS_DIR, safe_mkdir
from datetime import datetime

# === CONFIG ===
SCHEMA = "masterdatabase"
EXPORT_DIR = DATABASE_EXPORTS_DIR
safe_mkdir(EXPORT_DIR)


# ============================================================
# 📊 DEFINICIONES SQL DE LAS VIEWS
# ============================================================

VIEW_1_MASTER_JOIN_OPERATIONAL = f"""
CREATE OR REPLACE VIEW {SCHEMA}.master_join_operational AS
WITH fpi_exploded AS (
    SELECT
        farmer_number,
        UNNEST(contract_codes) AS contract_code,
        representative,
        phone,
        email,
        address,
        shipping_address,
        contract_name
    FROM {SCHEMA}.farmer_personal_information
)
SELECT
    COALESCE(fpi.farmer_number, 'NO_FARMER') AS farmer_number,
    COALESCE(fpi.contract_code, cti.contract_code, ca.contract_code, imc.contract_code) AS contract_code,

    -- FPI fields
    fpi.representative,
    fpi.phone,
    fpi.email,
    fpi.address,
    fpi.shipping_address,
    fpi.contract_name,

    -- CTI fields
    cti.contract_amendment,
    cti.etp_year,
    cti.harvest_year,
    cti.land_location,
    cti.latitude,
    cti.longitude,
    cti.planted,
    cti.planting_obligation,
    cti.planting_year,
    cti.region,
    cti.species,
    cti.strain,
    cti.trees_contract,
    cti.loaded_at AS cti_loaded_at,
    cti.planting_date,
    cti.status,
    cti.filter,
    cti.land_location_pro,
    cti.etp_year_backup,

    -- CA fields
    ca.usa_allocation_pct,
    ca.usa_trees_contracted,
    ca.usa_trees_planted,
    ca.canada_allocation_pct,
    ca.canada_2017_trees,
    ca.canada_2017_allocation_pct,
    ca.canada_trees_contracted,
    ca.total_can_allocation,
    ca.loaded_at AS ca_loaded_at,
    ca.etp_type,
    ca.contracted_cop,
    ca.planted_cop,
    ca.contracted_etp,
    ca.planted_etp,
    ca.surviving_etp,
    ca.surviving_cop,

    -- IMC fields (métricas actuales - usando rn=1 para obtener solo la más reciente)
    imc.rel_path,
    imc.planting_year AS imc_planting_year,
    imc.tree_age,
    imc.inventory_year,
    imc.inventory_date,
    imc.survival,
    imc.tht_mean,
    imc.tht_std,
    imc.mht_mean,
    imc.mht_std,
    imc.mht_pct_of_target,
    imc.dbh_mean,
    imc.dbh_std,
    imc.noncom_dbh_count,
    imc.dbh_pct_of_target,
    imc.doyle_bf_mean,
    imc.doyle_bf_std,
    imc.doyle_bf_total,
    imc.projected_dbh,
    imc.projected_doyle_bf,
    imc.pkid,
    imc.progress,
    imc.total_trees,
    imc.mortality,
    imc.planting_date AS imc_planting_date,
    imc.type_of_metric

FROM fpi_exploded fpi
FULL OUTER JOIN {SCHEMA}.contract_tree_information cti
    ON fpi.contract_code = cti.contract_code
FULL OUTER JOIN {SCHEMA}.contract_allocation ca
    ON COALESCE(fpi.contract_code, cti.contract_code) = ca.contract_code
FULL OUTER JOIN (
    SELECT * FROM {SCHEMA}.inventory_metrics_current
    WHERE rn = 1
) imc
    ON COALESCE(fpi.contract_code, cti.contract_code, ca.contract_code) = imc.contract_code

ORDER BY farmer_number, contract_code;
"""


VIEW_2_JOIN_MASTER = f"""
CREATE OR REPLACE VIEW {SCHEMA}.join_master AS
SELECT
    farmer_number,
    contract_code,
    representative,
    contract_name,
    species,
    status
FROM {SCHEMA}.master_join_operational
WHERE farmer_number IS NOT NULL
  AND contract_code IS NOT NULL
ORDER BY farmer_number, contract_code;
"""


VIEW_3_MASTER_JOIN_EXPANDED = f"""
CREATE OR REPLACE VIEW {SCHEMA}.master_join_expanded AS
WITH fpi_exploded AS (
    SELECT
        farmer_number,
        UNNEST(contract_codes) AS contract_code,
        farmer_name,
        phone_number,
        email,
        address,
        municipality,
        state,
        country,
        postal_code,
        bank_name,
        bank_account,
        clabe,
        rfc,
        curp,
        ine,
        proof_of_address,
        bank_statement,
        created_at AS fpi_created_at,
        updated_at AS fpi_updated_at
    FROM {SCHEMA}.farmer_personal_information
)
SELECT
    COALESCE(fpi.farmer_number, 'NO_FARMER') AS farmer_number,
    COALESCE(fpi.contract_code, cti.contract_code, ca.contract_code, im.contract_code) AS contract_code,

    -- FPI fields
    fpi.farmer_name,
    fpi.phone_number,
    fpi.email,
    fpi.address,
    fpi.municipality,
    fpi.state,
    fpi.country,
    fpi.postal_code,
    fpi.bank_name,
    fpi.bank_account,
    fpi.clabe,
    fpi.rfc,
    fpi.curp,
    fpi.ine,
    fpi.proof_of_address,
    fpi.bank_statement,
    fpi.fpi_created_at,
    fpi.fpi_updated_at,

    -- CTI fields
    cti.project_name,
    cti.specie,
    cti.contract_start_date,
    cti.contract_end_date,
    cti.total_trees_contracted,
    cti.tree_density,
    cti.surface_contracted_ha,
    cti.contract_status,
    cti.created_at AS cti_created_at,
    cti.updated_at AS cti_updated_at,

    -- CA fields
    ca.total_planted,
    ca.planted_2019,
    ca.planted_2020,
    ca.planted_2021,
    ca.planted_2022,
    ca.planted_2023,
    ca.planted_2024,
    ca.planted_2025,
    ca.created_at AS ca_created_at,
    ca.updated_at AS ca_updated_at,

    -- IM fields (DESGLOSE COMPLETO - inventory_metrics)
    im.year AS im_year,
    im.total_trees AS im_total_trees,
    im.alive AS im_alive,
    im.deceased AS im_deceased,
    im.survival_rate AS im_survival_rate,
    im.mortality_rate AS im_mortality_rate,
    im.created_at AS im_created_at,
    im.updated_at AS im_updated_at

FROM fpi_exploded fpi
FULL OUTER JOIN {SCHEMA}.contract_tree_information cti
    ON fpi.contract_code = cti.contract_code
FULL OUTER JOIN {SCHEMA}.contract_allocation ca
    ON COALESCE(fpi.contract_code, cti.contract_code) = ca.contract_code
FULL OUTER JOIN {SCHEMA}.inventory_metrics im
    ON COALESCE(fpi.contract_code, cti.contract_code, ca.contract_code) = im.contract_code

ORDER BY farmer_number, contract_code, im_year;
"""


# ============================================================
# 🛠️ FUNCIONES DE GESTIÓN
# ============================================================

def create_views(engine):
    """Crea las 3 views en PostgreSQL."""
    views = {
        "master_join_operational": VIEW_1_MASTER_JOIN_OPERATIONAL,
        "join_master": VIEW_2_JOIN_MASTER,
        "master_join_expanded": VIEW_3_MASTER_JOIN_EXPANDED
    }

    print(f"\n{'='*60}")
    print("🔧 CREANDO VIEWS EN POSTGRESQL")
    print(f"{'='*60}\n")

    with engine.begin() as conn:
        for view_name, view_sql in views.items():
            try:
                conn.execute(text(view_sql))
                print(f"✅ View creada: {SCHEMA}.{view_name}")
            except Exception as e:
                print(f"❌ Error creando {view_name}: {e}")

    print(f"\n{'='*60}\n")


def export_view_to_excel(engine, view_name, output_filename=None):
    """Exporta una view a Excel."""
    if output_filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f"{view_name}_{timestamp}.xlsx"

    output_path = EXPORT_DIR / output_filename

    print(f"📊 Exportando view: {view_name}")

    query = f'SELECT * FROM {SCHEMA}.{view_name};'

    with engine.begin() as conn:
        df = pd.read_sql(text(query), conn)

    # Normalizar timestamps
    for col in df.select_dtypes(include=["datetimetz"]).columns:
        df[col] = df[col].dt.tz_localize(None)

    df.to_excel(output_path, index=False)

    print(f"✅ Exportado: {output_path}")
    print(f"   Filas: {len(df):,}  |  Columnas: {len(df.columns)}\n")